In [ ]:
import pandas as pd
import numpy as np
import os
import sqlite3
import shutil
import datetime
import matplotlib.pyplot as plt
from itertools import combinations

In [ ]:
single_region = 0
timeslice = 0
elec_only = 0
remove_elec = 0

if single_region ==1:
    file_prefix = '../../PowerGenome/OEO_national/OEO_national_settings_'
else:
    if elec_only==0:
        file_prefix = '../../PowerGenome/OEO_regional/OEO_regional_settings_'
    else:
        file_prefix = '../../PowerGenome/OEO_regional_elec/OEO_regional_settings_elec_'

In [ ]:
# Create New SQL File
emptydB = 'US_BASE.sqlite'
#outfilename_w_ext = outFilename + '.sqlite'
if (single_region==1) & (timeslice==1) :
    outputdB = '../US_1R_TS.sqlite'
elif (single_region==1) & (timeslice==0) :
    outputdB = '../US_1R_4D.sqlite'
elif elec_only ==1:
    outputdB = '../US_9R_24D_elec.sqlite'
elif remove_elec ==1:
    outputdB = 'US_Regional_noelec.sqlite'
elif (single_region==0) & (timeslice==1):
    outputdB = '../US_9R_TS.sqlite'
else:
    outputdB = '../US_9R_8D.sqlite'

In [ ]:
all_periods = list(np.arange(2020,2051,5))
start_year = all_periods[0]

In [ ]:
# if elec_only==1:
#     elec_demand = pd.DataFrame()
#     for year in all_periods:
#         load_curves_year = make_final_load_curves(pudl_engine, scenario_settings[year][scenario])
#         load_curves_year.loc[:,'periods']=year
#         elec_demand = pd.concat([elec_demand, load_curves_year])

In [ ]:
#based on NREL ReEDS documentation, 2020: https://www.nrel.gov/docs/fy21osti/78195.pdf, Table 10 onwards
generator_lifetimes = dict()
# generator_lifetimes['BECCS'] = 50
generator_lifetimes['BIO'] = 50
generator_lifetimes['Batt'] = 15
generator_lifetimes['COALIGCC'] = 75
generator_lifetimes['COALSTM'] = 75
generator_lifetimes['COALUSC'] = 75
generator_lifetimes['GEO'] = 30
generator_lifetimes['H2'] = 60
generator_lifetimes['HYD'] = 100
generator_lifetimes['E_NGAACT'] = 50
generator_lifetimes['E_NGACT'] = 50
generator_lifetimes['E_NGAACC'] = 60
generator_lifetimes['E_NGACC'] = 60
generator_lifetimes['E_NGASTM'] = 75
generator_lifetimes['WND'] = 30 
generator_lifetimes['SOL'] = 30
generator_lifetimes['TRANS'] = 60
generator_lifetimes['URN'] = 70

In [ ]:
all_gens = pd.read_csv(file_prefix + 'all_gens.csv')
new_gen = pd.read_csv(file_prefix + 'new_gen.csv')

gen_variability = pd.read_csv(file_prefix + 'gen_variability.csv')

load_curves = pd.read_csv(file_prefix + 'load_curves.csv')
if '_4D' in outputdB:
    reduced_load_profile = pd.read_csv(file_prefix + 'reduced_load_profile_2P_2D.csv')
    time_series_mapping = pd.read_csv(file_prefix + 'time_series_mapping_2P_2D.csv')
    reduced_resource_profile = pd.read_csv(file_prefix + 'reduced_resource_2P_2D.csv')
elif '_24D' in outputdB:
    reduced_load_profile = pd.read_csv(file_prefix + 'reduced_load_profile_4P_6D.csv')
    time_series_mapping = pd.read_csv(file_prefix + 'time_series_mapping_4P_6D.csv')
    reduced_resource_profile = pd.read_csv(file_prefix + 'reduced_resource_4P_6D.csv')
elif '_21D' in outputdB:
    reduced_load_profile = pd.read_csv(file_prefix + 'reduced_load_profile_3P_7D.csv')
    time_series_mapping = pd.read_csv(file_prefix + 'time_series_mapping_3P_7D.csv')
    reduced_resource_profile = pd.read_csv(file_prefix + 'reduced_resource_3P_7D.csv')
elif '_14D' in outputdB:
    reduced_load_profile = pd.read_csv(file_prefix + 'reduced_load_profile_2P_7D.csv')
    time_series_mapping = pd.read_csv(file_prefix + 'time_series_mapping_2P_7D.csv')
    reduced_resource_profile = pd.read_csv(file_prefix + 'reduced_resource_2P_7D.csv')
elif '_12D' in outputdB:
    reduced_load_profile = pd.read_csv(file_prefix + 'reduced_load_profile_3P_4D.csv')
    time_series_mapping = pd.read_csv(file_prefix + 'time_series_mapping_3P_4D.csv')
    reduced_resource_profile = pd.read_csv(file_prefix + 'reduced_resource_3P_4D.csv')
elif '_8D' in outputdB:
    reduced_load_profile = pd.read_csv(file_prefix + 'reduced_load_profile_2P_4D.csv')
    time_series_mapping = pd.read_csv(file_prefix + 'time_series_mapping_2P_4D.csv')
    reduced_resource_profile = pd.read_csv(file_prefix + 'reduced_resource_2P_4D.csv')

if single_region!=1:
    if os.path.exists(file_prefix + 'transmission.csv'):
        transmission = pd.read_csv(file_prefix + 'transmission.csv')

In [ ]:
map_tech_names = dict()
map_tech_names['biomass'] = 'E_BIO_R'
map_tech_names['conventional_hydroelectric'] = 'E_HYDCONV_R'
map_tech_names['conventional_steam_coal'] = 'E_COALSTM_N'
map_tech_names['natural_gas_fired_combined_cycle'] = 'E_NGACC_R'
map_tech_names['natural_gas_fired_combustion_turbine'] = 'E_NGACT_R'
map_tech_names['natural_gas_steam_turbine'] = 'E_NGASTM_R'
map_tech_names['nuclear'] = 'E_URNLWR_R'
map_tech_names['onshore_wind_turbine'] = 'E_WND_R'
map_tech_names['small_hydroelectric'] = 'E_HYDSM_R'
map_tech_names['solar_photovoltaic'] = 'E_SOLPV_R'
map_tech_names['hydroelectric_pumped_storage'] = 'E_HYDPS_R'
map_tech_names['geothermal'] = 'E_GEO_R'
map_tech_names['naturalgas_ccccsavgcf_conservative'] = 'E_NGACC_CCS_N'
map_tech_names['naturalgas_ccavgcf_moderate'] = 'E_NGAACC_N'
map_tech_names['naturalgas_ctavgcf_moderate'] = 'E_NGAACT_N'
map_tech_names['landbasedwind_class4_moderate'] = 'E_WND_N'
map_tech_names['utilitypv_class1_moderate'] = 'E_SOLPVCEN_N'
map_tech_names['naturalgas_ccs100_moderate'] = 'E_NGACC_CCS_ZERO_N'
map_tech_names['nuclear_mid'] = 'E_URNLWR_N'
map_tech_names['battery_moderate'] = 'E_Batt'
map_tech_names['ev_load_shifting'] = 'E_LD_SHFT'
map_tech_names['offshorewind_class10_moderate'] = 'E_OFWND_N'
map_tech_names['geothermal_hydroflash'] = 'E_GEOF_N'
map_tech_names['geothermal_hydrobinary'] = 'E_GEOB_N'
map_tech_names['csp_class5_10hourstes'] = 'E_SOLTHCEN_N'
map_tech_names['biopower_dedicated_moderate'] = 'E_BIO_N'
map_tech_names['coal_igccavgcf_moderate'] = 'E_COALIGCC_N'
map_tech_names['coal_ccs90avgcf_moderate'] = 'E_COALUSC_90CCS_N'
map_tech_names['respv_class3'] = 'E_SOLPVENDUSE_N'

In [ ]:
map_tech_desc = dict()
map_tech_desc['E_BIO_R'] = '#existing bio-energy'
map_tech_desc['E_BIO_N'] = '#new bio-energy'
map_tech_desc['E_HYDSM_R'] = '#existing small hydroelectric power plant'
map_tech_desc['E_HYDPS_R'] = '#existing pumped hydro storage'
map_tech_desc['E_WND_N'] = '#new wind power plant'
map_tech_desc['E_NGACC_CCS_ZERO_N'] = '#new natural gas combined cycle with 100% CCS power plant'
map_tech_desc['E_OFWND_N'] = '#new offshore wind, floating'
map_tech_desc['E_GEOF_N'] = '#new geothermal, hydro flash'
map_tech_desc['E_GEOB_N'] = '#new geothermal, hydro binary'
map_tech_desc['E_COALUSC_90CCS_N'] = '#new ultrasupercritical pulverized coal with 90% CCS power plant'

In [ ]:
#map PG technology names to OEO names
all_gens['Resource'] = all_gens['Resource'].str.split('_').map(lambda x: '_'.join(x[0:3] if x[-1].isnumeric() else x))
new_gen['Resource'] = new_gen['Resource'].str.split('_').map(lambda x: '_'.join(x[0:3] if x[-1].isnumeric() else x))
all_gens['Resource'] = all_gens['Resource'].map(map_tech_names)
new_gen['Resource'] = new_gen['Resource'].map(map_tech_names)

In [ ]:
#multiply capex values by the regional multipliers, since this is not done in PowerGenome

new_gen['capex_mw'] *= new_gen['regional_cost_multiplier']
all_gens['capex_mw'] *= all_gens['regional_cost_multiplier']

In [ ]:
include_co2_trans = 0

In [ ]:
conn = sqlite3.connect(emptydB)
c = conn.cursor()
tech_table = pd.read_sql_query("SELECT * FROM technologies", conn)
tech_table.rename(columns={'tech':'tech_technologies'}, inplace=True)

input_table = pd.read_sql_query("SELECT input_comm, tech FROM Efficiency WHERE tech in \
(SELECT tech FROM technologies where sector='electric')", conn)
input_table = {k: list(v) for k,v in input_table.drop_duplicates().groupby('tech')["input_comm"]}
conn.close()


#input comms for technologies not specified in US_National
input_table['E_BIO_R'] = ['HERB_BIO']
input_table['E_BIO_N'] = input_table['E_BIOIGCC_N'] 
input_table['E_HYDPS_R'] = ['ELC']
input_table['E_HYDSM_R'] = ['ethos_R']
input_table['E_COALUSC_90CCS_N'] = ['COALIGCC_N'] #COALSTM_N
input_table['E_COALIGCC_N'] = ['COALIGCC_N']#COALSTM_N
input_table['E_NGACC_CCS_ZERO_N'] = ['E_NGA']
input_table['E_GEOB_N'] = ['ethos_R']
input_table['E_GEOF_N'] = ['ethos_R']
input_table['E_WND_N'] = ['ethos_R']
input_table['E_OFWND_N'] = ['ethos_R']
input_table['E_TRANS_R'] = ['ELC']
input_table['E_TRANS_N'] = ['ELC']
if include_co2_trans==1:
    input_table['co2_TRANS_N'] = ['co2_to_ground']

#emissionsactivity for technologies not specified in US_National 


In [ ]:
cluster_count = all_gens[['technology', 'cluster']].groupby('technology').max().reset_index()
cluster_count.columns = ['technology', 'max_cluster']
all_gens = all_gens.merge(cluster_count, on='technology', how='left')

mask = all_gens['max_cluster']<=1
all_gens.loc[mask, 'tech'] =  all_gens.loc[mask, 'Resource'] + '-' + all_gens.loc[mask,'region']
all_gens.loc[~mask, 'tech'] =  all_gens.loc[~mask, 'Resource'] + '-' + all_gens.loc[~mask,'region'] + \
'-' +all_gens.loc[~mask,'cluster'].map(int).map(str)

In [ ]:
cluster_count = new_gen[['technology', 'cluster']].groupby('technology').max().reset_index()
cluster_count.columns = ['technology', 'max_cluster']
new_gen = new_gen.merge(cluster_count, on='technology', how='left')

mask = new_gen['max_cluster']<=1
new_gen.loc[mask, 'tech'] =  new_gen.loc[mask, 'Resource'] + '-' + new_gen.loc[mask,'region']
new_gen.loc[~mask, 'tech'] =  new_gen.loc[~mask, 'Resource'] + '-' + new_gen.loc[~mask,'region'] + \
'-' +new_gen.loc[~mask,'cluster'].map(int).map(str)

In [ ]:
#concat start year gens with remaining new gens
all_gens_multi_year = pd.concat([all_gens, new_gen])

all_gens_multi_year = all_gens_multi_year.merge(tech_table, left_on = ['Resource'], right_on =['tech_technologies'], how='left')
all_gens_multi_year['flag'].fillna('p', inplace=True)
all_gens_multi_year['sector'].fillna('electric', inplace=True)
all_gens_multi_year['tech_desc'].fillna(all_gens_multi_year['Resource'].map(map_tech_desc), inplace=True)

all_gens_multi_year.loc[all_gens_multi_year['Resource']=='E_HYDPS_R','flag'] = 'ps'

#all_gens_multi_year = all_gens_multi_year[all_gens.columns]
all_gens_multi_year.loc[np.isnan(all_gens_multi_year.operating_year),'operating_year'] = all_periods[0]
all_gens_multi_year.loc[all_gens_multi_year.operating_year==0,'operating_year'] = all_periods[0]

In [ ]:
#account for storage (batteries) capex and FOM for a specific storage duration
stor_dur = 4
mask = all_gens_multi_year['tech'].str.contains('Batt')
all_gens_multi_year.loc[mask, 'Heat_Rate_MMBTU_per_MWh'] = 3.412/0.85 #assuming 85% efficiency
orig_batt = all_gens_multi_year.loc[mask,:].copy()
all_gens_multi_year.loc[mask,'capex_mw'] += stor_dur*all_gens_multi_year.loc[mask,'capex_mwh']
all_gens_multi_year.loc[mask,'Fixed_OM_Cost_per_MWyr'] += stor_dur*all_gens_multi_year.loc[mask,'Fixed_OM_Cost_per_MWhyr']
#all_gens_multi_year.loc[mask,'Fixed_OM_Cost_per_MWyr'] = 2.5/100*all_gens_multi_year.loc[mask,'capex_mwh']
stor_dur = 8
orig_batt.loc[mask,'capex_mw'] += stor_dur*orig_batt.loc[mask,'capex_mwh']
orig_batt.loc[mask,'Fixed_OM_Cost_per_MWyr'] += stor_dur*orig_batt.loc[mask,'Fixed_OM_Cost_per_MWhyr']
#orig_batt.loc[mask,'Fixed_OM_Cost_per_MWyr'] = 2.5/100*orig_batt.loc[mask,'capex_mwh']
orig_batt['Resource'] = 'E_Batt8hr'
orig_batt['tech'] = orig_batt['tech'].str.replace('E_Batt','E_Batt8hr')
orig_batt['tech_technologies'] = orig_batt['tech_technologies'].str.replace('E_Batt','E_Batt8hr')
orig_batt['tech_desc'] = orig_batt['tech_desc'].str.replace('storage','storage (8 hour)')

all_gens_multi_year = pd.concat([all_gens_multi_year, orig_batt])

In [ ]:
#duplicate NGCC costs, efficiencies to represent H2CC
mask = all_gens_multi_year['tech'].str.contains('E_NGAACC_N')
orig_ngcc = all_gens_multi_year.loc[mask,:].copy()
orig_ngcc.loc[:,'tech_desc'] = '#hydrogen combustion in a combined cycle plants for electricity generation'
orig_ngcc.loc[:, 'tech_technologies'] = orig_ngcc.loc[:, 'tech_technologies'].str.replace('NGAA', 'H2')
orig_ngcc.loc[:, 'tech'] = orig_ngcc.loc[:, 'tech'].str.replace('NGAA', 'H2')
orig_ngcc.loc[:, 'Resource'] = 'H2_100'

all_gens_multi_year = pd.concat([all_gens_multi_year, orig_ngcc])

In [ ]:
#create capacity factor dataframe
if timeslice==1:
    df_capfac_all= gen_variability.reset_index(drop=True)
    df_capfac_all.columns = all_gens.loc[:,'tech'] #rename columns to match df_gen technologies
    df_capfac_all.loc[:,'hour'] = np.tile(np.arange(0,24),365)
    df_capfac_all.loc[((df_capfac_all['hour']>=6) & (df_capfac_all['hour']<12)), 'time_of_day_name'] = 'am'
    df_capfac_all.loc[((df_capfac_all['hour']>=12) & (df_capfac_all['hour']<15)), 'time_of_day_name'] = 'peak'
    df_capfac_all.loc[((df_capfac_all['hour']>=15) & (df_capfac_all['hour']<21)), 'time_of_day_name'] = 'pm'
    df_capfac_all.loc[((df_capfac_all['hour']>=21) | (df_capfac_all['hour']<6)), 'time_of_day_name'] = 'night'
    summer = np.arange(171*24, 265*24) #June 20 to Sep 22
    winter = np.concatenate((np.arange(0,79*24), np.arange(355*24,8760))) #dec 21st#march 20th 
    df_capfac_all.loc[:, 'season_name'] = 'Intermediate'
    df_capfac_all.loc[summer, 'season_name'] = 'Summer'
    df_capfac_all.loc[winter, 'season_name'] = 'Winter'
    df_capfac = df_capfac_all.groupby(['time_of_day_name', 'season_name']).mean().reset_index()
    df_capfac.drop(columns=['hour'], inplace=True)
else:
    df_capfac= reduced_resource_profile.reset_index(drop=True)
    df_capfac.columns = all_gens.loc[:,'tech'] #rename columns to match df_gen technologies

In [ ]:
#remove solar thermal from all regions except CA and SW
#gens
mask = (all_gens_multi_year['Resource'].str.contains('E_SOLTH')) & \
((~all_gens_multi_year['tech'].str.contains('-CA')) & (~all_gens_multi_year['tech'].str.contains('-SW')))
all_gens_multi_year = all_gens_multi_year.loc[~mask,:]

In [ ]:
def return_region(df_tech):
    return [x[1] for x in df_tech.str.split('-')]

In [ ]:
def return_tech(df_tech):
    output_vals = []
    for val in df_tech:
        try:
            o_val = val.split('-')[0] + '_' + val.split('-')[2]
        except:
            o_val = val.split('-')[0]
        output_vals.append(o_val)
    return output_vals

In [ ]:
reg_mult = dict()
reg_mult['E_BECCS_N'] = new_gen.loc[new_gen['Resource']=='E_COALUSC_90CCS_N', ['region','regional_cost_multiplier']].drop_duplicates().set_index('region')
reg_mult['H2_STO150'] = new_gen.loc[new_gen['Resource']=='E_Batt', ['region','regional_cost_multiplier']].drop_duplicates().set_index('region')

In [ ]:

regions_list = all_gens_multi_year.region.unique()

# Delete old *.sqlite file (if it already exists) and copy/rename copy of temoa_schema.sqlite
if os.path.isfile(outputdB):
    os.remove(outputdB)
shutil.copyfile(emptydB, outputdB)
##remove data from tables
conn = sqlite3.connect(outputdB)
c = conn.cursor()

table_list = c.execute("SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'Output%'").fetchall()
c.execute("UPDATE Efficiency SET tech = TRIM(tech);") #trim spaces. Need to trim carriage return

#remove entire table
for table in [#'LifetimeProcess', #'CapacityToActivity'
              'CapacityCredit', 'CapacityFactorTech', 'MinGenGroupWeight','MinGenGroupTarget', 'MinCapacity',#'StorageDuration',
              #'GrowthRateMax','GrowthRateSeed', #'MinActivity',
               'time_periods','tech_curtailment',
              'Output_CapacityByPeriodAndTech','Output_V_Capacity','Output_VFlow_In', 'Output_VFlow_Out',
              'Output_Objective', 'Output_Emissions', 'Output_curtailment', 'Output_Costs',
               'tech_groups', 'groups', 'MyopicBaseyear', 'SegFrac','DemandSpecificDistribution'
              ]:
    query = """DELETE FROM """ + table
    c.execute(query)
    
if timeslice!=1:
    for table in ['time_of_day', 'time_season']:
        query = """DELETE FROM """ + table
        c.execute(query)
        
if single_region==0:
    query = "UPDATE EmissionLimit SET regions='global'"
    #query = "DELETE FROM EmissionLimit"
    c.execute(query)
   

#delete distributed generation (from solar) from Efficiency table in original database
#query = """DELETE FROM Efficiency WHERE input_comm='ELCDIST_R'"""
#c.execute(query)
query = """DELETE FROM Efficiency WHERE tech='IMPELC'"""
c.execute(query)

#removing upstream 'clean coal pathway, with pre-combustion retrofits'
query = """DELETE FROM Efficiency WHERE tech='E_CCR_COALSTM_N'"""
c.execute(query)

#deleting variable costs associated with transmission
#query = """DELETE FROM CostVariable WHERE tech='E_ELCTDLOSS'"""
#c.execute(query)

#modify max capacity geo techs
tech_keep = 'E_GEOBCFS_N'
df = pd.read_sql_query("SELECT * FROM MaxCapacity WHERE tech='" + tech_keep + "'", conn)
df1 = df.copy()
df2 = df.copy()
df1['tech'] = 'E_GEOF_N'
df2['tech'] = 'E_GEOB_N'
query = "DELETE FROM MaxCapacity WHERE tech='" + tech_keep + "'"
c.execute(query)
df_new = pd.concat([df1, df2])
df_new.to_sql('MaxCapacity',conn, if_exists='append', index=False)


techs_keep = ['E_BECCS_N','H2_STO150', 'E_BECCS_N_emissions']#, 'E_H2CC_N']
old_df_c2a = pd.read_sql("SELECT * FROM CapacityToActivity", conn)
#query = "DELETE FROM CapacityToActivity WHERE tech NOT IN ('" + "','".join(techs_keep) + "')"
#c.execute(query)
for tech_keep in techs_keep:
    #modify the technologies that are retained from US_National for multiple regions
    for table in table_list:
        df_cols = c.execute("SELECT * FROM pragma_table_info('" + table[0] + "')").fetchall()
        df_cols = [x[1] for x in df_cols]
        if 'tech' in df_cols:
            df = pd.read_sql_query("SELECT * FROM " + table[0] + " WHERE tech='" + tech_keep + "'", conn)
            if table[0]=='CapacityFactorTech':
                query = "DELETE FROM " + table[0] + " WHERE tech='" + tech_keep + "'"
                c.execute(query)
            elif (len(df)>0) & ('regions' in df.columns):
                query = "DELETE FROM " + table[0] + " WHERE tech='" + tech_keep + "'"
                c.execute(query)
                for reg in regions_list:
                    df_new = df.copy()
                    df_new['regions'] = reg
                    if table[0]=='CostInvest':
                        df_new['cost_invest'] *= reg_mult[tech_keep].loc[reg].values[0]
                    df_new.to_sql(table[0],conn, if_exists='append', index=False)
        
#modify efficiency for processes that meet demands that can only be met by processes of vintage 2017
for table in ['Efficiency','EmissionActivity']:
    query = """UPDATE """ + table + """ SET vintage=2020
    WHERE vintage=2017
    AND regions || input_comm || tech || output_comm NOT IN 
    (
        SELECT DISTINCT
        regions || input_comm || tech || output_comm
        FROM """ + table + """ WHERE vintage = 2020
    )"""
    c.execute(query)

#modify cost tables to account for these processes
for table in ['CostFixed','CostVariable']:
    query= """UPDATE """ + table + """ SET vintage=2020
        WHERE 
          vintage = 2017 
          AND
          regions || tech NOT IN 
        (SELECT DISTINCT
          regions || tech
        FROM """ + table + """ WHERE vintage = 2020)"""
    c.execute(query)

#delete all elec technologies except for those in techs_keep
for table in ['Efficiency', 'ExistingCapacity', 'DiscountRate', 'CostVariable', 'CostFixed', 'CostInvest']:
    query = "DELETE FROM " + table + " WHERE tech IN\
    (SELECT tech FROM technologies WHERE sector='electric')\
    AND tech NOT IN ('" + "','".join(techs_keep) + "')"
    c.execute(query)

#delete efficiencies from 2017 vintages
for table in ['Efficiency','EmissionActivity']:
    query = """DELETE FROM """ + table + """ WHERE vintage=2017"""
    c.execute(query)


table = 'EmissionLimit'
query = """DELETE FROM """ + table + """ WHERE periods=2017"""
c.execute(query)    
        
# Delete row from Efficiency if (t,v) retires at the begining of crent period (which is time_periods[i][0])
c.execute("DELETE FROM Efficiency WHERE tech IN (SELECT tech FROM LifetimeProcess WHERE \
             LifetimeProcess.life_process+LifetimeProcess.vintage<=2020) \
             AND vintage IN (SELECT vintage FROM LifetimeProcess WHERE LifetimeProcess.life_process+\
             LifetimeProcess.vintage<=2020);")

# Delete row from Efficiency if (t,v) retires at the begining of crent period (which is time_periods[i][0])
c.execute("DELETE FROM Efficiency WHERE tech IN (SELECT tech FROM LifetimeTech WHERE \
             LifetimeTech.life+Efficiency.vintage<=2020) AND \
             vintage NOT IN (SELECT vintage FROM LifetimeProcess WHERE LifetimeProcess.tech\
             =Efficiency.tech);")

# If row is not deleted via the last two DELETE commands, it might still be invalid for period
#  time_periods[i][0] since they can have model default lifetime of 40 years. 
c.execute("DELETE FROM Efficiency WHERE tech IN (SELECT tech FROM Efficiency WHERE \
            40+Efficiency.vintage<=2020) AND \
            tech NOT IN (SELECT tech FROM LifetimeTech) AND \
            vintage NOT IN (SELECT vintage FROM LifetimeProcess WHERE LifetimeProcess.tech=Efficiency.tech);")


#remove demands, techinput split before 2020
for table in ['Demand','TechInputSplit', 'MinActivity', 'MaxActivity','CostFixed','CostVariable']:
    query = """DELETE FROM """ + table +""" where periods < 2020"""
    c.execute(query)
for table in ['CostFixed','CostVariable']:
    query = """DELETE FROM """ + table +""" where vintage=2017"""
    c.execute(query)
for table in ['CostInvest', 'DiscountRate']:
    query = """DELETE FROM """ + table + """ where vintage < 2020"""
    c.execute(query)
    
#if creating electricity only database, delete all other demands
if elec_only==1:
    query = "DELETE FROM Efficiency WHERE output_comm IN (SELECT demand_comm FROM Demand WHERE demand_comm IS NOT 'dummy_sink')"
    c.execute(query)
    c.execute("DELETE FROM Demand WHERE demand_comm IS NOT 'dummy_sink'")
    c.execute("DELETE FROM DemandSpecificDistribution")
elif timeslice==1: 
    c.execute("DELETE FROM DemandSpecificDistribution")

conn.commit()
conn.close()


In [ ]:
def swap_regions(x):
    return x.split('-')[1] + '-' + x.split('-')[0]

In [ ]:
#remove coal and gas clusters with greater than 60% efficiency
mask = (all_gens_multi_year['Heat_Rate_MMBTU_per_MWh']<3.412/0.6) & \
((all_gens_multi_year['technology'].str.contains('Natural')) | (all_gens_multi_year['technology'].str.contains('Coal')))
all_gens_multi_year = all_gens_multi_year.loc[~mask]


In [ ]:
#check data for existing nuclear which is currently zero
all_gens_multi_year['Var_OM_Cost_per_MWh'].fillna(0, inplace=True)

In [ ]:
#def prep_tables(period, folder_name, gen_data, capfac_variability_data):
    #PREPARE DATABASE TABLES
    
df_gen= all_gens_multi_year
df_gen.rename(columns={'region':'regions'}, inplace=True)

#remove clusters with cap size of less than 100 MW

mask1 = (df_gen['tech'].str.contains('E_WND_N')) | (df_gen['tech'].str.contains('E_SOLPVCEN_N'))
mask2 = df_gen['Max_Cap_MW']<100
df_capfac = df_capfac.loc[:,~df_capfac.columns.isin(df_gen.loc[mask1 & mask2, 'tech'].drop_duplicates().values)]
df_gen = df_gen.loc[~(mask1 & mask2),:]

df_gen.loc[:,'regions'] = df_gen.loc[:,'regions'].str.replace('US_N','US')
#df_gen.loc[:, 'tech'] =  df_gen.loc[:, 'Resource'] + '-' + df_gen.loc[:,'regions'] + '-' +df_gen.loc[:,'cluster'].map(int).map(str)

#remove battery, pumped storage and ev_shifting (after column names have been assigned to capfac dataframe)
df_gen = df_gen.loc[~df_gen.loc[:,'tech'].str.contains('E_LD_SHFT'),:]
#df_gen = df_gen.loc[~df_gen.loc[:,'tech'].str.contains('battery'),:]
#df_gen = df_gen.loc[~df_gen.loc[:,'tech'].str.contains('pumped'),:]
#rename operating_year as vintage
df_gen.rename(columns={'operating_year':'vintage'},inplace=True)
df_gen= df_gen.astype({'vintage':int})

#modify lifetimes of clusters post-PG runs
df_gen.loc[:,'lifetime'] = [generator_lifetimes[y] for x in df_gen['tech'] for y in generator_lifetimes.keys() if y.lower() in x.lower()] #map lifetimes

#capacityfactortech
#remove ev_shifting
df_capfac.drop(columns=df_capfac.columns[df_capfac.columns.str.contains('E_LD_SHFT')], inplace=True)
#df_capfac.drop(columns=df_capfac.columns[df_capfac.columns.str.contains('battery')], inplace=True)
#df_capfac.drop(columns=df_capfac.columns[df_capfac.columns.str.contains('pumped')], inplace=True)

intra_annual_periods = len(df_capfac)
if timeslice==0:
    days = len(time_series_mapping['Rep_Period'].unique()) #intra_annual_periods/24
    iter_val = 0
    hours = int(len(df_capfac)/days)
    for day in np.arange(days)+1:
        df_capfac.loc[iter_val:iter_val+hours-1,'season_name'] = 'S' + str(int(day))
        df_capfac.loc[iter_val:iter_val+hours-1,'time_of_day_name'] = ['H' + str(int(x)) for x in np.arange(1,hours+1)]
        iter_val+=hours
df_capfac = df_capfac.melt(id_vars=['season_name','time_of_day_name'])
df_capfac = df_capfac.rename(columns={'variable':'tech', 'value':'cf_tech'})
df_capfac.loc[:,'cf_tech_notes'] = 'from PowerGenome'
df_capfac.insert(0,'regions',return_region(df_capfac.loc[:,'tech']))
#df_capfac.loc[:,'regions'] = df_capfac.loc[:,'regions'].str.replace('US_N','US')
df_capfac.loc[:,'tech'] = return_tech(df_capfac.loc[:,'tech'])
    
#remove techs that have constant capacity factor
df_sum = df_capfac.groupby(by=['regions','tech']).sum().reset_index()
df_fixed_capfac = df_sum[df_sum.cf_tech==intra_annual_periods]
df_fixed_capfac.rename(columns={'cf_tech': 'cf_tech_max'}, inplace=True)
df_capfac = df_capfac.merge(df_fixed_capfac, on = ['regions','tech'], how='outer')
df_capfac = df_capfac[df_capfac.cf_tech_max!=intra_annual_periods]
df_capfac = df_capfac.drop('cf_tech_max',axis=1)

#segfrac for a chronological model
if timeslice==0:
    df_segfrac = df_capfac.loc[:,['season_name','time_of_day_name']].copy()
    df_segfrac = df_segfrac.drop_duplicates().reset_index(drop=True)
    df_segfrac.loc[:,'segfrac'] = 0

    seg_frac_period_weights = time_series_mapping[['Period_Index','Rep_Period']].groupby(by='Rep_Period').sum()
    seg_frac_period_weights = seg_frac_period_weights['Period_Index']/seg_frac_period_weights['Period_Index'].sum()

    slot_periods = int(len(df_segfrac)/len(seg_frac_period_weights))
    slots = int(len(df_segfrac)/slot_periods)

    start_val = 0
    for i in np.arange(slots):
        mask = np.arange(start_val,start_val+slot_periods,1)
        df_segfrac.loc[mask, 'segfrac'] = seg_frac_period_weights.iloc[i]/slot_periods
        start_val += slot_periods

#identify renewables that qualify for RPS and hydro, but not storage 
df_gen.loc[:,'renewable_nonstor'] = df_gen.loc[:,'RPS'] | df_gen.loc[:,'HYDRO'] 
df_gen.loc[df_gen['technology'].str.contains('Wind'),'renewable_nonstor'] = 1
df_gen.loc[df_gen['technology'].str.contains('PV'),'renewable_nonstor'] = 1
df_gen.loc[df_gen['technology'].str.contains('CSP'),'renewable_nonstor'] = 1
df_gen.loc[df_gen['technology'].str.contains('Solar'),'renewable_nonstor'] = 1

#convert heat rates of all renewables to zero
df_gen.loc[df_gen['technology'].str.contains('Wind'),'Heat_Rate_MMBTU_per_MWh'] = 0
df_gen.loc[df_gen['technology'].str.contains('PV'),'Heat_Rate_MMBTU_per_MWh'] = 0
df_gen.loc[df_gen['technology'].str.contains('CSP'),'Heat_Rate_MMBTU_per_MWh'] = 0
df_gen.loc[df_gen['technology'].str.contains('Solar'),'Heat_Rate_MMBTU_per_MWh'] = 0
df_gen.loc[df_gen['technology'].str.contains('Geo'),'Heat_Rate_MMBTU_per_MWh'] = 0
df_gen.loc[df_gen['technology'].str.contains('Conventional Hydroelectric'),'Heat_Rate_MMBTU_per_MWh'] = 0
df_gen.loc[df_gen['technology'].str.contains('Small Hydroelectric'),'Heat_Rate_MMBTU_per_MWh'] = 0

#estimate retirement years based on lifetime and vintage
mask = df_gen.loc[:,'vintage']>0
df_gen.loc[mask, 'retirement_year'] = df_gen.loc[mask, 'vintage'] + df_gen.loc[mask, 'lifetime']

#update lifetime as difference between retirement year and operating year
mask = df_gen.loc[:,'vintage'] + df_gen.loc[:,'lifetime'] <=start_year
df_gen.loc[mask, 'lifetime'] = start_year - df_gen.loc[mask,'vintage'] + 5
#df_gen.loc[:,'lifetime_diff'] = df_gen.loc[:,'retirement_year'] - df_gen.loc[:,'vintage']
#df_gen.loc[:,'lifetime_diff'].fillna(0, inplace=True)

#df_gen.loc[:,'lifetime'].fillna(0, inplace=True)
#df_gen.loc[:,'lifetime'] = df_gen.loc[:, ['lifetime', 'lifetime_diff']].max(axis=1)

#capacitytoactivity for a chronological model, to normalize to annual time periods, assuming min value is hours
df_c2a = df_gen.loc[:,['tech']].drop_duplicates()
#remove battery and ev_shifting
df_c2a = df_c2a.loc[~df_c2a.loc[:,'tech'].str.contains('ev_'),:]
df_c2a = df_c2a.loc[~df_c2a.loc[:,'tech'].str.contains('battery'),:]
df_c2a = df_c2a.loc[~df_c2a.loc[:,'tech'].str.contains('pumped'),:]

df_c2a.loc[:,'c2a'] = 31.536 #8760/(days*24)
df_c2a.loc[:,'c2a_notes'] = 'from PowerGenome'
df_c2a.insert(0,'regions',return_region(df_c2a.loc[:,'tech']))
df_c2a.loc[:,'regions'] = df_c2a.loc[:,'regions'].str.replace('US_N','US')
df_c2a.loc[:,'tech'] = return_tech(df_c2a.loc[:,'tech'])

#investment, fixed and variable costs
df_costs = df_gen.loc[:,['tech', 'capex_mw', 'Fixed_OM_Cost_per_MWyr', 'Var_OM_Cost_per_MWh', 'vintage', 'interconnect_annuity', 'spur_inv_mwyr']].copy()
df_costs['interconnect_annuity'].fillna(0, inplace=True)
df_costs['spur_inv_mwyr'].fillna(0, inplace=True)
df_costs.loc[:,'cost_invest'] = df_costs.loc[:, 'capex_mw']/(10**3) #$/MW to #$M/GW
df_costs.loc[:,'cost_fixed'] = (df_costs.loc[:, 'Fixed_OM_Cost_per_MWyr'])*(10**3)/(10**6) #$/MWyr to #$M/GW-yr
df_costs.loc[:,'cost_variable'] = df_costs.loc[:, 'Var_OM_Cost_per_MWh']*(277777.78)/(10**6) #$/MWh to #$M/PJ
df_costs.loc[:,'cost_invest_units'] = '$M/GW'
df_costs.loc[:,'cost_invest_notes'] = 'from PowerGenome'
df_costs.loc[:,'cost_fixed_units'] = '$M/GWyr'
df_costs.loc[:,'cost_fixed_notes'] = 'from PowerGenome'
df_costs.loc[:,'cost_variable_units'] = '$M/PJ'
df_costs.loc[:,'cost_variable_notes'] = 'from PowerGenome'
df_costs.insert(0,'regions',return_region(df_costs.loc[:,'tech']))
df_costs.loc[:,'tech'] = return_tech(df_costs.loc[:,'tech'])

df_costs['cost_fixed'] += (df_costs.loc[:, 'interconnect_annuity'])*(10**3)/(10**6) #$/MW to #$M/GW #add spur annuity cost to capital costs

# df_costs['cost_invest'] += (df_costs.loc[:, 'interconnect_annuity'] + df_costs.loc[:, 'spur_inv_mwyr'])*(10**3)/(10**6) #$/MW to #$M/GW #add annuity and spur costs to capital costs
df_cost_invest = df_costs.loc[:,['regions','tech','vintage','cost_invest','cost_invest_units','cost_invest_notes']].copy()
df_cost_invest = df_cost_invest.loc[df_cost_invest.cost_invest>0]

df_costv = df_costs.loc[:,['regions','tech','vintage','cost_variable','cost_variable_units','cost_variable_notes']].drop_duplicates()
df_cost_variable = pd.DataFrame(np.repeat(df_costv.values, len(all_periods), axis=0), columns= df_costv.columns)
df_cost_variable.insert(1,'periods',int((len(df_cost_variable)/len(all_periods)))*all_periods)
df_cost_variable = df_cost_variable[df_cost_variable.vintage<=df_cost_variable.periods]

df_costf = df_costs.loc[:,['regions','tech','vintage','cost_fixed','cost_fixed_units','cost_fixed_notes']]
df_cost_fixed = pd.DataFrame(np.repeat(df_costf.values, len(all_periods), axis=0), columns= df_costf.columns)
df_cost_fixed.insert(1,'periods',int((len(df_cost_fixed)/len(all_periods)))*all_periods)
df_cost_fixed = df_cost_fixed[df_cost_fixed.vintage<=df_cost_fixed.periods]


#efficiency
df_efficiency = df_gen.loc[:,['Resource','tech', 'Heat_Rate_MMBTU_per_MWh','renewable_nonstor', 'vintage', 'flag', 'sector', 'tech_desc', 'tech_category']].copy()
df_efficiency.rename(columns={'Resource':'input_comm'}, inplace=True)
#convert technologies with no heat rate                               to 100% efficiency, by setting the heat rate to 3412.0/1000 MMBTU/MWh
df_efficiency.loc[(df_efficiency.loc[:,'Heat_Rate_MMBTU_per_MWh']==0),'Heat_Rate_MMBTU_per_MWh'] = 3412.0/1000
df_efficiency.loc[:, 'output_comm'] = df_efficiency.loc[:,'renewable_nonstor'].apply(lambda x: 'ELCP_Renewables' if x ==1 else 'ELCP')
mask = df_efficiency.loc[:,'flag']=='ps'
df_efficiency.loc[mask, 'input_comm'] = 'ELC'
df_efficiency.loc[mask, 'output_comm'] = 'ELC'
mask = (df_efficiency['tech'].str.contains('HYD')) & (df_efficiency['flag']!='ps')
df_efficiency.loc[mask, 'output_comm' ] = 'ELCP_Renewables_nonRPS'
df_efficiency.loc[:,'efficiency'] = 3412.0/(df_efficiency.loc[:,'Heat_Rate_MMBTU_per_MWh']*1000)
df_efficiency.loc[:,'efficiency'].fillna(1.0, inplace=True)
df_efficiency.drop(columns=['Heat_Rate_MMBTU_per_MWh','renewable_nonstor'], inplace=True)
df_efficiency.loc[:,'eff_notes'] = 'from PowerGenome'
#remove battery and ev_shifting
df_efficiency = df_efficiency.loc[~df_efficiency.loc[:,'tech'].str.contains('ev_'),:]
df_efficiency = df_efficiency.loc[~df_efficiency.loc[:,'tech'].str.contains('battery'),:]
df_efficiency.insert(0,'regions',return_region(df_efficiency.loc[:,'tech']))
df_efficiency.loc[:,'tech'] = return_tech(df_efficiency.loc[:,'tech'])

#ramp up and down fractions
df_ramp = df_gen.loc[(df_gen.loc[:,'Ramp_Up_percentage']>0) | (df_gen.loc[:,'Ramp_Dn_percentage']>0), ['regions','tech', 'Ramp_Up_percentage', 'Ramp_Dn_percentage']].copy()
df_ramp.rename(columns={'Ramp_Up_percentage': 'ramp_up', 'Ramp_Dn_percentage': 'ramp_down'}, inplace=True)
df_ramp.loc[:,'tech'] = return_tech(df_ramp.loc[:,'tech'])
df_ramp = df_ramp[(df_ramp.ramp_up<1) | (df_ramp.ramp_down<1)]

#lifetime
#df_lifetime = df_gen.loc[:,['tech','lifetime']].copy()
#df_lifetime.insert(0,'regions',return_region(df_lifetime.loc[:,'tech']))
#df_lifetime.loc[:,'tech'] = return_tech(df_lifetime.loc[:,'tech'])

#existing capacity
df_ex_cap = df_gen.loc[:, ['tech','Existing_Cap_MW', 'vintage' ]]
df_ex_cap = df_ex_cap.loc[df_ex_cap.loc[:,'Existing_Cap_MW']>0,:]
df_ex_cap.loc[:,'exist_cap'] = df_ex_cap.loc[:,'Existing_Cap_MW']/1000 #GW
df_ex_cap.drop(columns=['Existing_Cap_MW'], inplace=True)
df_ex_cap.loc[:,'exist_cap_units'] = 'GW'
df_ex_cap.loc[:,'exist_cap_notes'] = 'from PowerGenome'
##FIX THIS for zero value years for EV load shifting, removing this row for now
df_ex_cap = df_ex_cap.loc[df_ex_cap.loc[:,'vintage']!=0,:]
df_ex_cap.insert(0,'regions',return_region(df_ex_cap.loc[:,'tech']))
df_ex_cap.loc[:,'tech'] = return_tech(df_ex_cap.loc[:,'tech'])

#discount rate
df_wacc = df_gen.loc[(df_gen.loc[:,'wacc_real']>0), ['tech', 'vintage', 'wacc_real']].copy()
df_wacc.rename(columns={'wacc_real': 'tech_rate'}, inplace=True)
df_wacc.loc[:,'tech_rate_notes'] = 'from PowerGenome'
df_wacc.insert(0,'regions',return_region(df_wacc.loc[:,'tech']))
df_wacc.loc[:,'tech'] = return_tech(df_wacc.loc[:,'tech'])

#capital recovery years
df_cap_rec_years = df_gen.loc[(df_gen.loc[:,'cap_recovery_years']>0),['tech', 'cap_recovery_years']].copy()
df_cap_rec_years.rename(columns={'cap_recovery_years': 'loan'}, inplace=True)
df_cap_rec_years.loc[:,'loan_notes'] = 'from PowerGenome'
df_cap_rec_years.insert(0,'regions',return_region(df_cap_rec_years.loc[:,'tech']))
df_cap_rec_years.loc[:,'tech'] = return_tech(df_cap_rec_years.loc[:,'tech'])

#lifetime
df_lifetime = df_gen.loc[(df_gen.loc[:,'lifetime']>0),['tech', 'lifetime']].copy()
df_lifetime.rename(columns={'lifetime': 'life'}, inplace=True)
df_lifetime.loc[:,'life_notes'] = 'from NREL ReEDS 2020'
df_lifetime.insert(0,'regions',return_region(df_lifetime.loc[:,'tech']))
df_lifetime.loc[:,'tech'] = return_tech(df_lifetime.loc[:,'tech'])

#modify fixed and variable costs based on lifetime
df_cost_variable_merge = df_cost_variable.merge(df_lifetime, on=['regions','tech'])
df_cost_variable_merge['retirement_year'] = df_cost_variable_merge['vintage'] + df_cost_variable_merge['life']
df_cost_variable = df_cost_variable_merge[df_cost_variable_merge['retirement_year']>df_cost_variable_merge['periods']]
df_cost_variable = df_cost_variable.drop(['life','life_notes','retirement_year'], axis=1)

df_cost_fixed_merge = df_cost_fixed.merge(df_lifetime, on=['regions','tech'])
df_cost_fixed_merge['retirement_year'] = df_cost_fixed_merge['vintage'] + df_cost_fixed_merge['life']
df_cost_fixed = df_cost_fixed_merge[df_cost_fixed_merge['retirement_year']>df_cost_fixed_merge['periods']]
df_cost_fixed = df_cost_fixed.drop(['life','life_notes','retirement_year'], axis=1)

#capacity credit
#modify capacity credit for renewables
df_gen.loc[df_gen.tech.str.contains('WND'),'CapRes'] = 0.1
df_gen.loc[df_gen.tech.str.contains('SOLPVCEN'),'CapRes'] = 0.1
df_gen.loc[df_gen.tech.str.contains('SOLTHCEN_N'),'CapRes'] = 0.1
df_gen.loc[df_gen.tech.str.contains('SOLPV_R'),'CapRes'] = 0.1

df_ccredit = df_gen.loc[df_gen.loc[:,'CapRes']>0, ['tech', 'vintage', 'CapRes']].copy()
df_ccredit  = pd.DataFrame(np.repeat(df_ccredit.values, len(all_periods), axis=0), columns= df_ccredit.columns)
df_ccredit.insert(0,'periods',int((len(df_ccredit)/len(all_periods)))*all_periods)
df_ccredit.rename(columns={'CapRes': 'cf_tech'}, inplace=True)
df_ccredit.loc[:,'cf_tech_notes'] = 'from PowerGenome'
df_ccredit.insert(0,'regions',return_region(df_ccredit.loc[:,'tech']))
df_ccredit.loc[:,'tech'] = return_tech(df_ccredit.loc[:,'tech'])
mask = df_ccredit.loc[:,'periods']>=df_ccredit.loc[:,'vintage']
df_ccredit = df_ccredit.loc[mask,:]

#max capacity, for renewables, primarily
df_maxcap = df_gen.loc[(df_gen.loc[:,'Max_Cap_MW']>0),['tech','Max_Cap_MW','vintage']].copy()
df_maxcap.rename(columns={'Max_Cap_MW':'maxcap', 'vintage':'periods'}, inplace=True)
df_maxcap.loc[:,'maxcap'] = df_maxcap.loc[:,'maxcap']/1000 #convert MW to GW
df_maxcap.loc[:,'maxcap_units'] = 'GW' #convert MW to GW
df_maxcap.loc[:,'maxcap_notes'] = 'from PowerGenome'
df_maxcap.insert(0,'regions',return_region(df_maxcap.loc[:,'tech']))
df_maxcap.loc[:,'tech'] = return_tech(df_maxcap.loc[:,'tech'])

#load, estimating demand specific distribution, using data from EFS
if timeslice==0:
    df_load_i = reduced_load_profile
    ind = list(reduced_load_profile.columns).index('Time_Index')
    df_load_i = reduced_load_profile[reduced_load_profile.columns[ind+1:]]
    df_load_i = df_load_i.reset_index(drop=True).multiply(df_segfrac.loc[:,'segfrac'].reset_index(drop=True), axis=0)
    if elec_only!=1:
        df_load_i = df_load_i/df_load_i.sum()
    iter_val = 0
    for day in np.arange(days)+1:
        df_load_i.loc[iter_val:iter_val+hours-1,'season_name'] = 'S' + str(int(day))
        df_load_i.loc[iter_val:iter_val+hours-1,'time_of_day_name'] = ['H' + str(int(x)) for x in np.arange(1,hours+1)]
        iter_val+=hours
    df_load_i = df_load_i.melt(id_vars=['season_name','time_of_day_name'])
    df_load_i = df_load_i.rename(columns={'variable':'regions', 'value':'dds'})

    df_load = pd.DataFrame()
    if elec_only!=1:
        #demns = ['RSC', 'RSH','RLT','ROELC','RWH','CLT','COELC','COEELC','CSC','CSH','CWH']
        df_load_i.loc[:, 'demand_name'] = df_load_i.loc[:, 'regions'].map(lambda x: x.split('_')[-1])
        if single_region==1:
            df_load_i.loc[:, 'regions'] = 'US'
        else:
            df_load_i.loc[:, 'regions'] = df_load_i.loc[:, 'regions'].map(lambda x: x.replace('_' + x.split('_')[-1],''))
        df_load_i.loc[:, 'regions'] = df_load_i.loc[:, 'regions'].str.replace('US_N','US')
        df_load = df_load_i[['regions','season_name','time_of_day_name','demand_name','dds']]
    else:
        demn = 'ELC_dem'
        for region in df_load_i.regions.unique():
            df_load_i.loc[(df_load_i.regions==region),'demand_name'] = demn
            df_load_i.loc[(df_load_i.regions==region),'dds'] = df_load_i.loc[(df_load_i.regions==region),'dds']/df_load_i.loc[(df_load_i.regions==region),'dds'].sum()
            df_load = pd.concat([df_load, df_load_i.loc[(df_load_i.regions==region),['regions','season_name','time_of_day_name','demand_name','dds']]])
else:
    df_load_i= load_curves.reset_index(drop=True)
    df_load_i.loc[:,'hour'] = np.tile(np.arange(0,24),365)
    df_load_i.loc[((df_load_i['hour']>=6) & (df_load_i['hour']<12)), 'time_of_day_name'] = 'am'
    df_load_i.loc[((df_load_i['hour']>=12) & (df_load_i['hour']<15)), 'time_of_day_name'] = 'peak'
    df_load_i.loc[((df_load_i['hour']>=15) & (df_load_i['hour']<21)), 'time_of_day_name'] = 'pm'
    df_load_i.loc[((df_load_i['hour']>=21) | (df_load_i['hour']<6)), 'time_of_day_name'] = 'night'
    summer = np.arange(171*24, 265*24) #June 20 to Sep 22
    winter = np.concatenate((np.arange(0,79*24), np.arange(355*24,8760))) #dec 21st#march 20th 
    df_load_i.loc[:, 'season_name'] = 'Intermediate'
    df_load_i.loc[summer, 'season_name'] = 'Summer'
    df_load_i.loc[winter, 'season_name'] = 'Winter'
    df_load = df_load_i.groupby(['time_of_day_name', 'season_name']).sum().reset_index()
    df_load.drop(columns=['hour'], inplace=True)
    df_load.loc[:,'regions'] = 'US'

    df_load = df_load.melt(id_vars=['time_of_day_name','season_name','regions'],value_name='dds')
    df_load.rename(columns={'variable':'demand_name'} , inplace=True)
    df_load = df_load.merge(df_load.groupby(by=['demand_name']).sum().reset_index(), on = ['demand_name'])
    df_load['dds'] = df_load['dds_x']/df_load['dds_y']
    if single_region==0:
        df_load.loc[:,'regions'] = df_load['demand_name'].str.split('_').apply(lambda x: '_'.join(x[0:-1]))
        df_load.loc[:,'demand_name'] = df_load['demand_name'].str.split('_').apply(lambda x: x[-1])
    else:
        df_load.loc[:,'regions'] = 'US'
    df_load = df_load.loc[:,['regions','season_name','time_of_day_name','demand_name','dds']]

    df_segfrac = df_load_i.iloc[:,-3:].reset_index(drop=True)
    df_segfrac.drop(columns=['hour'], inplace=True)
    df_segfrac['segfrac'] = 1
    df_segfrac = df_segfrac.groupby(by=['time_of_day_name','season_name']).sum().reset_index()
    df_segfrac['segfrac'] = df_segfrac['segfrac']/df_segfrac['segfrac'].sum()
    df_segfrac.loc[:,'segfrac_notes'] = 'from PowerGenome'  

df_load.loc[:,'dds_notes'] = 'from EFS and PowerGenome'  

model_regions = df_efficiency['regions'].unique()

if os.path.exists(file_prefix + 'transmission.csv'):

    #transmission efficiency
    df_trans_efficiency = pd.DataFrame(columns = df_efficiency.columns)
    df_trans_efficiency.loc[:,'regions']  = transmission.loc[:,'transmission_path_name'].str.replace('_to_','-')
    df_trans_efficiency.loc[:,'input_comm'] = 'ELC'
    df_trans_efficiency.loc[:,'tech'] = 'E_TRANS_R'
    df_trans_efficiency.loc[:,'vintage'] = all_periods[0]-1
    df_trans_efficiency.loc[:,'output_comm'] = 'ELC'
    df_trans_efficiency.loc[:,'efficiency'] = 1 - transmission.loc[:,'Line_Loss_Percentage']
    df_trans_efficiency.loc[:,'eff_notes'] = 'from PowerGenome'
    df_trans_efficiency.loc[:,'flag'] = 'p'
    df_trans_efficiency.loc[:,'sector'] = 'electric'
    df_trans_efficiency.loc[:,'tech_desc'] = '#electricity transmission'
    
    df_efficiency = pd.concat([df_efficiency, df_trans_efficiency])
    df_trans_efficiency['regions'] = df_trans_efficiency['regions'].apply(swap_regions)
    df_efficiency = pd.concat([df_efficiency, df_trans_efficiency])
    df_trans_efficiency_n= df_trans_efficiency.copy()
    
    #transmission_efficiency_new
    df_trans_efficiency = pd.DataFrame(columns = df_efficiency.columns)
    df_trans_efficiency.loc[:,'regions']  = transmission.loc[:,'transmission_path_name'].str.replace('_to_','-')
    df_trans_efficiency.loc[:,'vintage'] = all_periods[0]
    df_trans_efficiency.loc[:,'input_comm'] = 'ELC'
    df_trans_efficiency.loc[:,'tech'] = 'E_TRANS_N'
    df_trans_efficiency.loc[:,'output_comm'] = 'ELC'
    df_trans_efficiency.loc[:,'efficiency'] = 1 - transmission.loc[:,'Line_Loss_Percentage']
    df_trans_efficiency.loc[:,'eff_notes'] = 'from PowerGenome'
    df_trans_efficiency.loc[:,'flag'] = 'p'
    df_trans_efficiency.loc[:,'sector'] = 'electric'
    df_trans_efficiency.loc[:,'tech_desc'] = '#new electricity transmission'

    df_trans_efficiency_r = pd.DataFrame(np.repeat(df_trans_efficiency.values, len(all_periods), axis=0), columns= df_trans_efficiency.columns)
    df_trans_efficiency_r.loc[:,'vintage'] = np.tile(all_periods, len(df_trans_efficiency))
    
    df_efficiency = pd.concat([df_efficiency, df_trans_efficiency_r])
    df_trans_efficiency_r['regions'] = df_trans_efficiency_r['regions'].apply(swap_regions)
    df_efficiency = pd.concat([df_efficiency, df_trans_efficiency_r])

    #transmission investment costs 
    df_trans_c = pd.DataFrame(columns = df_cost_invest.columns)
    df_trans_c.loc[:,'regions']  = transmission.loc[:,'transmission_path_name'].str.replace('_to_','-')
    df_trans_c.loc[:,'vintage'] = all_periods[0]
    df_trans_c.loc[:,'tech'] = 'E_TRANS_N'
    #df_trans_c.loc[:,'vintage'] = all_periods[0]-1
    df_trans_c.loc[:, 'cost_invest'] = transmission.loc[:, 'Line_Reinforcement_Cost_per_MWyr']*(10**3)/(10**6) #$/MW to #$M/GW
    df_trans_c.loc[:, 'cost_invest_units'] = '$M/GW'
    df_trans_c.loc[:, 'cost_invest_notes'] = 'from PowerGenome'
    df_trans_costs = pd.DataFrame(np.repeat(df_trans_c.values, len(all_periods), axis=0), columns= df_trans_c.columns)
    df_trans_costs.loc[:,'vintage'] = np.tile(all_periods, len(df_trans_c))
    
    df_cost_invest = pd.concat([df_cost_invest, df_trans_costs])
    df_trans_costs['regions'] = df_trans_costs['regions'].apply(swap_regions)
    df_cost_invest = pd.concat([df_cost_invest, df_trans_costs])
    
    #transmission investment wacc
    df_trans_wacc = df_trans_costs[['regions', 'tech', 'vintage']].copy().drop_duplicates()
    df_trans_wacc.loc[:,'tech_rate'] = 0.069 #from PG example system settings file
    df_trans_wacc.loc[:, 'tech_rate_notes'] = 'from PowerGenome'

    df_wacc = pd.concat([df_wacc, df_trans_wacc])
    df_trans_wacc['regions'] = df_trans_wacc['regions'].apply(swap_regions)
    df_wacc = pd.concat([df_wacc, df_trans_wacc])
    
    #new transmission lifetime
    df_trans_lifetime = df_trans_c[['regions', 'tech']].copy().drop_duplicates()
    df_trans_lifetime.loc[:,'life'] = 60 #from PG example system settings file
    df_trans_lifetime.loc[:, 'life_notes'] = 'from PowerGenome'

    df_lifetime = pd.concat([df_lifetime, df_trans_lifetime])
    df_trans_lifetime['regions'] = df_trans_lifetime['regions'].apply(swap_regions)
    df_lifetime = pd.concat([df_lifetime, df_trans_lifetime])

    #trans existing capacity
    df_trans_cap = pd.DataFrame()

    df_trans_cap.loc[:,'regions']  = transmission.loc[:,'transmission_path_name'].str.replace('_to_','-')
    df_trans_cap.loc[:,'vintage']  = all_periods[0]-1

    df_trans_cap.loc[:,'exist_cap'] = abs(transmission.loc[:,['Line_Max_Flow_MW', 'Line_Min_Flow_MW']]).max(axis=1).values/1000

    df_trans_cap.insert(1,'tech', 'E_TRANS_R')
    df_trans_cap.loc[:,'exist_cap_units'] = 'GW'
    df_trans_cap.loc[:,'exist_cap_notes'] = 'from PowerGenome'
    
    df_ex_cap = pd.concat([df_ex_cap, df_trans_cap])
    df_trans_cap['regions'] = df_trans_cap['regions'].apply(swap_regions)
    df_ex_cap = pd.concat([df_ex_cap, df_trans_cap])
    
    #capacity2activity
    df_trans_gen = pd.concat([df_trans_efficiency_r, df_trans_efficiency_n])
    df_c2a_trans = df_trans_gen[['regions','tech']].drop_duplicates()

    df_c2a_trans.loc[:,'c2a'] = 31.536 #8760/(days*24)
    df_c2a_trans.loc[:,'c2a_notes'] = 'from PowerGenome'
    df_c2a = pd.concat([df_c2a, df_c2a_trans])
    
    #add co2 transport between regions to efficiency
    if include_co2_trans==1:
        df_co2_transport = df_trans_efficiency_r.copy()
        df_co2_transport['efficiency']  = 0.99
        df_co2_transport['tech']  = 'co2_TRANS_N'
        df_co2_transport['sector'] = 'supply'
        df_co2_transport['eff_notes'] = 'allows co2 transport between adjoining regions'
        df_co2_transport['tec_desc'] = 'new inter-region co2 transport pipelines'
        df_co2_transport['input_comm'] = 'co2_to_ground'
        df_co2_transport['output_comm'] = 'co2_to_ground'
        df_co2_transport_tx = df_co2_transport[df_co2_transport['regions'].str.contains('TX')].copy()
        for regs in ['SW', 'SE']:
            df_co2_transport_mod = df_co2_transport_tx.copy()
            df_co2_transport_mod['regions'] = df_co2_transport_mod['regions'].str.replace('CEN',regs)
            df_co2_transport = pd.concat([df_co2_transport, df_co2_transport_mod ])
        df_efficiency = pd.concat([df_efficiency, df_co2_transport])
        df_co2_transport['regions'] = df_co2_transport['regions'].apply(swap_regions)
        df_efficiency = pd.concat([df_efficiency, df_co2_transport])


#size of individual units in new technologies cluster
df_technology_new_cluster = df_gen.loc[df_gen['Existing_Cap_MW'].isna(),['tech', 'Cap_size']]
df_technology_new_cluster.insert(0,'regions' ,return_region(df_technology_new_cluster.loc[:,'tech']))
df_technology_new_cluster['tech'] =  return_tech(df_technology_new_cluster.loc[:,'tech'])
df_technology_new_cluster.rename(columns={'Cap_size':'cap_size'}, inplace=True)
df_technology_new_cluster['cap_size'] /= 1000 #convert MW to GW

In [ ]:
#add input_comm from commodities list
tech_wo_clusters = df_efficiency['tech'].map(lambda x: '_'.join(x.split('_')[0:-1]) if x.split('_')[-1].isnumeric() else x)
df_efficiency.loc[:,'input_comm'] = tech_wo_clusters.map({k: v[0] for k,v in input_table.items()})

#duplicate bio to have multiple inputs
bio = df_efficiency[df_efficiency.loc[:,'tech']=='E_BIO_N']
bio.loc[:, 'input_comm'] = input_table['E_BIO_N'][1]
df_efficiency = pd.concat([df_efficiency, bio])
bio.loc[:, 'input_comm'] = input_table['E_BIO_N'][2]
df_efficiency = pd.concat([df_efficiency, bio])

In [ ]:
#add nox and SO2 emissions to other electric power plants
#technologies affected:
conn = sqlite3.connect(emptydB)
c = conn.cursor()
techs_so2 = ['E_NGASTM_R', 'E_NGACT_R']
df_emiss = pd.read_sql("SELECT * FROM EmissionActivity WHERE emis_comm='so2_ELC' AND tech in ('" + "','".join(list(techs_so2)) +\
                       "' )", conn)
#query = "DELETE FROM EmissionActivity WHERE emis_comm='co2' AND tech in \
#('" + "','".join(list(techs_so2)) + "' )"
emis_group = df_emiss.groupby('tech').min()

df_new_emiss = pd.DataFrame()
for tech in techs_so2:
    emis_techs = df_efficiency.loc[df_efficiency['tech'].str.contains(tech), :]
    emis_techs.loc[:,'emis_act'] = emis_group.loc[emis_group.index.str.contains(tech), 'emis_act'].values[0]
    emis_techs['emis_comm'] = 'so2_ELC'
    emis_techs['emis_act_units'] = 'kt/PJout'
    emis_techs['emis_act_notes'] = 'from original national db'
    df_new_emiss = pd.concat([df_new_emiss,emis_techs ])

In [ ]:
techs_nox = ['E_NGASTM_R', 'E_NGACT_R', 'E_NGACC_R', 'E_NGACC_CCS_N']

df_emiss = pd.read_sql("SELECT * FROM EmissionActivity WHERE emis_comm='nox_ELC' AND tech in ('" + "','".join(list(techs_nox)) +\
                       "' )", conn)
emis_group = df_emiss.groupby('tech').min()
for tech in techs_nox:
    emis_techs = df_efficiency.loc[df_efficiency['tech'].str.contains(tech), :]
    emis_techs.loc[:,'emis_act'] = emis_group.loc[emis_group.index.str.contains(tech), 'emis_act'].values[0]
    emis_techs['emis_comm'] = 'nox_ELC'
    emis_techs['emis_act_units'] = 'kt/PJout'
    emis_techs['emis_act_notes'] = 'from original national db'
    if tech=='E_NGACC_CCS_N':
        emis_techs['tech'] = 'E_NGACC_CCS_ZERO_N'
    df_new_emiss = pd.concat([df_new_emiss,emis_techs ])

In [ ]:
#add CO2 output commodity to the efficiency tables
# co2 emissions factors from here: https://www.eia.gov/tools/faqs/faq.php?id=73&t=11
emiss_dict = {}
emiss_dict['COALSTM_N'] = 88.42 #using the min of what's in the db at this time 205.7/2.2/(9.478e+5)*1000*1000 # lbs/MMBTU to kt/PJ
emiss_dict['E_NGA'] = 50 # using the min of what's in the db 117.0/2.2/(9.478e+5)*1000*1000 # lbs/MMBTU to kt/PJ
emiss_dict['COALIGCC_N'] = 88.42 #using the min of what's in the db at this time 205.7/2.2/(9.478e+5)*1000*1000 # lbs/MMBTU to kt/PJ

mask = df_efficiency['tech'].str.contains('CCS')
ccs_techs = df_efficiency.loc[mask, :].copy()
ccs_techs_emissions = df_efficiency.loc[mask, :].copy()
ccs_techs.loc[:,'fuel_emis'] = ccs_techs['input_comm'].map(emiss_dict)
ccs_techs.loc[:,'input_comm'] = 'ethos'
ccs_techs.loc[:,'output_comm'] = 'co2_CCS'
ccs_techs.loc[:,'tech'] = ccs_techs.loc[:,'tech'] + '_emissions'

ccs_techs.loc[:,'co2_removal_eff'] = 0.9
ccs_techs.loc[(ccs_techs.loc[:,'tech'].str.contains('ZERO')),'co2_removal_eff'] = 1
ccs_techs.loc[:, 'eff_notes'] = 'associated dummy process to account for co2 as a physical commodity'

#ccs_techs.loc[:,'to_split'] = 1 - ccs_techs.loc[:,'efficiency']/( ccs_techs.loc[:,'efficiency']+ (ccs_techs.loc[:,'co2_removal_eff']*ccs_techs.loc[:,'fuel_emis']))

#modify the outputs of all ccs techs to be PJ + co2 emissions
#df_efficiency.loc[mask,'efficiency'] = ccs_techs.loc[:,'efficiency']+  ccs_techs.loc[:,'co2_removal_eff']*ccs_techs.loc[:,'fuel_emis']
ccs_techs.loc[:,'efficiency'] = 1 #ccs_techs.loc[:,'co2_removal_eff']*ccs_techs.loc[:,'fuel_emis']

df_efficiency = pd.concat([df_efficiency, ccs_techs[df_efficiency.columns]])

# add CO2 output to the emissionsactivity tables

ccs_techs_emissions.loc[:,'fuel_emis'] = ccs_techs_emissions['input_comm'].map(emiss_dict)
ccs_techs_emissions.loc[:,'co2_removal_eff'] = 0.9
ccs_techs_emissions.loc[(ccs_techs_emissions.loc[:,'tech'].str.contains('ZERO')),'co2_removal_eff'] = 1
ccs_techs_emissions.insert(1,'emis_comm','co2')
ccs_techs_emissions['emis_act'] = -ccs_techs_emissions.loc[:,'co2_removal_eff']*ccs_techs_emissions.loc[:,'fuel_emis']/ccs_techs_emissions.loc[:,'efficiency']
ccs_techs_emissions['emis_act_units'] = 'kt/PJout'
ccs_techs_emissions['emis_act_notes'] = 'negative emissions from capturing the carbon'
#ccs_techs['to_split_notes'] = 'TechOutputSplit for ccs techs  is calculated as (A/(A+B)) or (B/(A+B)) where A and B are the original efficiencies for the technology  co2_ccs and ELCP (or ELCP_Renewables)'
ccs_techs_emissions['periods'] = ccs_techs_emissions['vintage']

conn = sqlite3.connect(outputdB)
c = conn.cursor()
df_emiss = pd.read_sql("SELECT * FROM EmissionActivity WHERE emis_comm='co2' AND tech in ('" + "','".join(list(ccs_techs_emissions.tech.unique())) +\
                       "' )", conn)
query = "DELETE FROM EmissionActivity WHERE emis_comm='co2' AND tech in \
('" + "','".join(list(ccs_techs_emissions.tech.unique())) + "' )"
c.execute(query)


#update lifetimes
mask = df_lifetime['tech'].str.contains('CCS')
ccs_tech_lifetimes = df_lifetime.loc[mask, :].copy()
ccs_tech_lifetimes.loc[:,'tech'] = ccs_tech_lifetimes.loc[:,'tech'] + '_emissions'
df_lifetime = pd.concat([df_lifetime, ccs_tech_lifetimes])

#update linked techs
df_linkedtechs = ccs_techs_emissions[['regions','tech']].drop_duplicates()
df_linkedtechs.columns = 'primary_' + df_linkedtechs.columns.str.replace('regions','region')
df_linkedtechs['linked_tech'] = df_linkedtechs['primary_tech'] + '_emissions'
df_linkedtechs.insert(2,'emis_comm','co2')
df_linkedtechs['tech_linked_notes'] = 'from PowerGenome'
query = "DELETE FROM LinkedTechs WHERE primary_tech IN ('" + "','".join(df_linkedtechs['primary_tech'].values) + "')" 
c.execute(query)
conn.commit()
conn.close()

In [ ]:
#change existing wind and solar efficiencies to 1
df_efficiency.loc[(df_efficiency.loc[:,'tech'].str.contains('wind')) & (df_efficiency.loc[:,'vintage']<start_year), 'efficiency'] = 1
df_efficiency.loc[(df_efficiency.loc[:,'tech'].str.contains('solar')) & (df_efficiency.loc[:,'vintage']<start_year), 'efficiency'] = 1

In [ ]:
#change output_comm for residential PV to ELCDIST
df_efficiency.loc[df_efficiency.loc[:,'tech']=='E_SOLPVENDUSE_N', 'output_comm'] = 'ELCDIST_R'
respv = df_efficiency.loc[df_efficiency.loc[:,'tech']=='E_SOLPVENDUSE_N',:]

for reg in respv['regions'].unique():
    
    #pick the 5 largest clusters
    mask1 = df_maxcap['regions']==reg
    mask2 = df_maxcap['tech'].str.contains('E_SOLPVCEN')
    df_solar_maxcap = df_maxcap.loc[mask1 & mask2 & (df_maxcap['periods']==start_year)]
    df_solar_maxcap = df_solar_maxcap.nlargest(5,'maxcap')

    mask1 = df_capfac['regions']==reg
    mask2 = df_capfac['tech'].str.contains('E_SOLPVCEN')
    df_capfac_sol_reg = df_capfac[mask1 & mask2]
    df_capfac_sol_reg = df_capfac_sol_reg.merge(df_segfrac.drop_duplicates())
    df_capfac_sol_reg['weighted_cf'] = df_capfac_sol_reg['cf_tech']*df_capfac_sol_reg['segfrac']
    
    #pick the cluster with the highest CF
    df_solar = df_capfac_sol_reg.groupby(by='tech')['weighted_cf'].sum().reset_index()
    df_solar = df_solar[df_solar['tech'].isin(df_solar_maxcap['tech'])]
    sel_solar_tech = df_solar.loc[df_solar['weighted_cf'].idxmax()]['tech']

    respv = df_capfac.loc[(df_capfac['regions']==reg) & (df_capfac['tech']==sel_solar_tech)]
    respv.loc[:,'tech']= 'E_SOLPVENDUSE_N'
    
    df_capfac = pd.concat([df_capfac, respv])

In [ ]:
#add min residential PV generation to MinGenGroupWeight table
df_target = pd.DataFrame()
df_gen_respv = pd.read_csv('electric/Residential_PV_gen_GWh.csv')
if single_region==1:
    df_gen_respv = df_gen_respv.groupby(['year']).sum().reset_index()
    df_gen_respv['reg_names'] = 'US'
for reg in model_regions:
    df_reg = df_gen_respv[df_gen_respv.reg_names==reg]
    df_reg = df_reg[['reg_names', 'year', 'value']]
    df_reg['value'] *= 0.0036 #convert GWh to PJ
    df_target = pd.concat([df_target, df_reg ])
    
df_target['value'] = np.round(df_target['value'],2)
df_target = df_target.rename(columns={'year':'periods','reg_names':'regions','value':'minact'})
df_target.insert(2,'tech','E_SOLPVENDUSE_N')
df_target.insert(4,'minact_units','PJ')

df_target.loc[:,'minact_notes'] = 'From NREL dGen model mid PV cost scenario'

In [ ]:
df_blend_target = df_target.copy()
df_blend_target['maxact'] = df_blend_target['minact']*1.2
df_blend_target = df_blend_target[['regions','periods','tech','maxact']]
df_blend_target['maxact_units'] = 'PJ'
df_blend_target['maxact_notes'] = 'From NREL dGen model mid PV cost scenario, 120%'

In [ ]:
#add capacity factor data for solar thermal technologies
csp = df_capfac.loc[df_capfac.loc[:,'tech']=='E_SOLPVCEN_N_1',:]
csp.loc[:,'tech'] = csp.loc[:,'tech'].str.replace('E_SOLPVCEN_N_1', 'E_SOLTHCEN_N')
csp = csp[(csp['regions']=='CA') | (csp['regions']=='SW')] #solar thermal only in CA and SW
csp.loc[:,'cf_tech'] = 0.64
df_capfac = pd.concat([df_capfac, csp])

In [ ]:
#add RPS constraints
df_rps = pd.read_csv('electric/FEDERAL_Updated Regional Renewable Portfolio Standards 2000-2050.csv')
df_rps.set_index('Region', inplace=True)
df_rps = df_rps.loc[:,[str(x) for x in all_periods]]
df_rps = df_rps[df_rps.index!='Federal']

region_names = pd.read_csv('region_names.csv', header=None, index_col=0)
region_names = region_names.to_dict()[1]

df_rps.index = [region_names[int(x)] for x in df_rps.index]
df_rps = df_rps.apply(lambda x: x.str.replace('%', '').astype(float)/100, axis=1)
df_rps = df_rps.reset_index().melt(id_vars='index')
df_rps = df_rps.rename(columns={'index':'regions', 'variable':'periods', 'value':'ti_split'})
df_rps.insert(2, 'input_comm','ELCP_Renewables')
df_rps.insert(3, 'tech','E_ELCTDLOSS')
df_rps.loc[:,'ti_split_notes'] = 'Average approximate RPS representation'

df_rps = df_rps[df_rps['regions'].isin(model_regions)]

In [ ]:
#storage duration to existing pumped hydro
df_stordur = df_efficiency[(df_efficiency['flag']=='ps') & (df_efficiency['tech'].str.contains('HYD'))]
df_stordur = df_stordur[['regions','tech']].drop_duplicates()
if len(df_stordur)>0:
    df_stordur.loc[:, 'duration'] = 10 #assuming 10 hours
    df_stordur.loc[:, 'duration_notes'] = 'assuming 10 hours for all pumped hydro storage'

In [ ]:
#add 90% availability factor to nuclear power plants
df_capfac_nuclear = df_capfac[(df_capfac['tech']==df_capfac.iloc[0]['tech']) & (df_capfac['regions']==df_capfac.iloc[0]['regions'])]
df_capfac_nuclear.loc[:,'cf_tech'] = 0.9
df_capfac_nuclear.loc[:,'cf_tech_notes'] = 'EPA IPM assumptions'
df_capfac_nuclear_all = pd.DataFrame()
for ind, row in df_efficiency.loc[df_efficiency['tech'].str.contains('URN'),['regions','tech']].iterrows():
    df_capfac_nuclear.loc[:,'tech'] = row['tech']
    df_capfac_nuclear.loc[:,'regions'] = row['regions']
    df_capfac_nuclear_all = pd.concat([df_capfac_nuclear_all, df_capfac_nuclear])
df_capfac = pd.concat([df_capfac, df_capfac_nuclear_all])

In [ ]:
#add 10% ITC to centralized solar pv 
mask = df_cost_invest['tech'].str.contains('E_SOLPVCEN')
df_cost_invest.loc[mask, 'cost_invest'] *=0.9

In [ ]:
#WRITE TO DATABASE
#also replace all the e_coalstm_n techs by e_coalstm_r techs
conn = sqlite3.connect(outputdB)

#CapacityToActivity
df_c2a = df_c2a.drop_duplicates()
df_table = df_c2a[~df_c2a['tech'].isin(old_df_c2a['tech'])]
df_table['tech'] = df_table['tech'].str.replace('E_COALSTM_N_', 'E_COALSTM_R_')
sqlite_table = 'CapacityToActivity'
df_table.to_sql(sqlite_table, conn, if_exists='append', index=False)

#SegFrac
# if timeslice==0:
df_table = df_segfrac.drop_duplicates()
sqlite_table = 'SegFrac'
df_table.to_sql(sqlite_table, conn, if_exists='append', index=False)

#CostInvest
df_table = df_cost_invest.copy()
df_table = df_table.loc[df_table.loc[:,'cost_invest']>0,:]
df_table['tech'] = df_table['tech'].str.replace('E_COALSTM_N_', 'E_COALSTM_R_')
sqlite_table = 'CostInvest'
df_table.to_sql(sqlite_table, conn, if_exists='append', index=False)

#CostVariable
df_table = df_cost_variable.drop_duplicates()
df_table['tech'] = df_table['tech'].str.replace('E_COALSTM_N_', 'E_COALSTM_R_')
sqlite_table = 'CostVariable'
df_table.to_sql(sqlite_table, conn, if_exists='append', index=False)

#CostFixed
df_table = df_cost_fixed.drop_duplicates()
df_table['tech'] = df_table['tech'].str.replace('E_COALSTM_N_', 'E_COALSTM_R_')
sqlite_table = 'CostFixed'
df_table.to_sql(sqlite_table, conn, if_exists='append', index=False)

#Efficiency
df_table = df_efficiency[['regions', 'input_comm','tech','vintage','output_comm', 'efficiency', 'eff_notes']].drop_duplicates()
df_table['tech'] = df_table['tech'].str.replace('E_COALSTM_N_', 'E_COALSTM_R_')
sqlite_table = 'Efficiency'
df_table.to_sql(sqlite_table, conn, if_exists='append', index=False)

#tech_curtailment
df_table = pd.DataFrame(df_efficiency.loc[['WND' in x or 'PV' in x or 'SOL' in x for x in df_efficiency.loc[:,'tech']],'tech'].drop_duplicates())
df_table = df_table[~df_table['tech'].str.contains('BLND')]
df_table.insert(1,'notes','')
sqlite_table = 'tech_curtailment'
df_table.to_sql(sqlite_table, conn, if_exists='append', index=False)

#tech_production in technologies table
df_table = df_efficiency[['tech', 'flag', 'sector', 'tech_desc', 'tech_category']].drop_duplicates()
df_table = df_table[~df_table['tech'].isin(tech_table['tech_technologies'])]
df_table['tech'] = df_table['tech'].str.replace('E_COALSTM_N_', 'E_COALSTM_R_')
sqlite_table = 'technologies'
df_table.to_sql(sqlite_table, conn, if_exists='append', index=False)

#tech_ramping
df_table = df_ramp.loc[:,['tech']].drop_duplicates()
df_table['tech'] = df_table['tech'].str.replace('E_COALSTM_N_', 'E_COALSTM_R_')
sqlite_table = 'tech_ramping'
if timeslice!=1:
    df_table.to_sql(sqlite_table, conn, if_exists='append', index=False)

#ramp up
df_table = df_ramp.loc[:,['regions','tech', 'ramp_up']].drop_duplicates()
df_table['tech'] = df_table['tech'].str.replace('E_COALSTM_N_', 'E_COALSTM_R_')
sqlite_table = 'RampUp'
if timeslice!=1:
    df_table.to_sql(sqlite_table, conn, if_exists='append', index=False)

#ramp down
df_table = df_ramp.loc[:,['regions','tech', 'ramp_down']].drop_duplicates()
df_table['tech'] = df_table['tech'].str.replace('E_COALSTM_N_', 'E_COALSTM_R_')
sqlite_table = 'RampDown'
if timeslice!=1:
    df_table.to_sql(sqlite_table, conn, if_exists='append', index=False)

#ExistingCapacity
df_table = df_ex_cap.drop_duplicates()
df_table['tech'] = df_table['tech'].str.replace('E_COALSTM_N_', 'E_COALSTM_R_')
sqlite_table = 'ExistingCapacity'
df_table.to_sql(sqlite_table, conn, if_exists='append', index=False)

#DiscountRate
df_table = df_wacc.loc[:,['regions','tech', 'vintage', 'tech_rate', 'tech_rate_notes']].drop_duplicates()
df_table['tech'] = df_table['tech'].str.replace('E_COALSTM_N_', 'E_COALSTM_R_')
sqlite_table = 'DiscountRate'
df_table.to_sql(sqlite_table, conn, if_exists='append', index=False)

#LifetimeLoanTech
df_table = df_cap_rec_years.drop_duplicates()
df_table['tech'] = df_table['tech'].str.replace('E_COALSTM_N_', 'E_COALSTM_R_')
sqlite_table = 'LifetimeLoanTech'
df_table.to_sql(sqlite_table, conn, if_exists='append', index=False)

#LifetimeTech
c = conn.cursor()
df_lifetime_current = pd.read_sql_query("SELECT * FROM LifetimeTech", conn)
nonoverlap_tech = df_lifetime_current[~df_lifetime_current['tech'].isin(df_lifetime['tech'])]
query = "DELETE FROM LifetimeTech"
c.execute(query)
#df_lifetime.loc[:,'life']=120 #all to 120 years for now

df_table = pd.concat([nonoverlap_tech, df_lifetime.drop_duplicates()])
df_table['tech'] = df_table['tech'].str.replace('E_COALSTM_N_', 'E_COALSTM_R_')
sqlite_table = 'LifetimeTech'
df_table.to_sql(sqlite_table, conn, if_exists='append', index=False)

#CapacityCredit
df_table = df_ccredit.loc[:,['regions','periods','tech', 'vintage','cf_tech','cf_tech_notes']].drop_duplicates()
df_table['tech'] = df_table['tech'].str.replace('E_COALSTM_N_', 'E_COALSTM_R_')
sqlite_table = 'CapacityCredit'
df_table.to_sql(sqlite_table, conn, if_exists='append', index=False)

#tech_reserve
df_table = df_ccredit.loc[:,['tech','cf_tech_notes']].drop_duplicates(subset='tech')
df_table = df_table[~df_table['tech'].isin(tech_table['tech_technologies'])]
df_table.rename(columns={'cf_tech_notes':'notes'}, inplace=True)
df_table['tech'] = df_table['tech'].str.replace('E_COALSTM_N_', 'E_COALSTM_R_')
sqlite_table = 'tech_reserve'
df_table.to_sql(sqlite_table, conn, if_exists='append', index=False)

#CapacityFactorTech
df_table = df_capfac.drop_duplicates()
df_table['tech'] = df_table['tech'].str.replace('E_COALSTM_N_', 'E_COALSTM_R_')
df_table.loc[df_table['cf_tech']<0.01,'cf_tech']=0
df_table.loc[df_table['cf_tech']>1,'cf_tech']=1

sqlite_table = 'CapacityFactorTech'
df_table.to_sql(sqlite_table, conn, if_exists='append', index=False)

#write_sql(df_table, sqlite_table, outputdB)

#MaxCapacity
df_table = df_maxcap.loc[:,['regions','periods','tech', 'maxcap','maxcap_units','maxcap_notes']].drop_duplicates()
df_table['tech'] = df_table['tech'].str.replace('E_COALSTM_N_', 'E_COALSTM_R_')
sqlite_table = 'MaxCapacity'
df_table.to_sql(sqlite_table, conn, if_exists='append', index=False)

#DSD
if timeslice==0:
    df_table = df_load.loc[:,['regions','season_name','time_of_day_name','demand_name','dds','dds_notes']]
    sqlite_table = 'DemandSpecificDistribution'
    df_table.to_sql(sqlite_table, conn, if_exists='append', index=False)

    #time_season
    df_table = pd.DataFrame(df_capfac.loc[:,'season_name'].drop_duplicates())
    df_table.rename(columns={'season_name':'t_season'},inplace=True)
    sqlite_table = 'time_season'
    df_table.to_sql(sqlite_table, conn, if_exists='append', index=False)

    #time_of_day
    df_table = pd.DataFrame(df_capfac.loc[:,'time_of_day_name'].drop_duplicates())
    df_table.rename(columns={'time_of_day_name':'t_day'},inplace=True)
    sqlite_table = 'time_of_day'
    df_table.to_sql(sqlite_table, conn, if_exists='append', index=False)
else:
    df_table = df_load
    sqlite_table = 'DemandSpecificDistribution'
    df_table.to_sql(sqlite_table, conn, if_exists='append', index=False)

#time_periods
df_table = pd.DataFrame(columns=['t_periods','flag'])
#df_table.loc[:,'t_periods'] = np.sort(df_costs.loc[:,'vintage'].unique().astype(int))
df_table.loc[:,'t_periods'] = np.hstack((np.arange(df_costs.loc[:,'vintage'].astype(int).min()-1,2020), 
                                         np.arange(2020,2056,5)))
df_table.loc[:,'flag'] = ['e' if x <2020  else 'f' for x in df_table.loc[:,'t_periods']]
sqlite_table = 'time_periods'
df_table.to_sql(sqlite_table, conn, if_exists='append', index=False)

# #MinGenGroupWeight
# df_table = df_weights
# df_table['tech'] = df_table['tech'].str.replace('E_COALSTM_N_', 'E_COALSTM_R_')
# sqlite_table = 'MinGenGroupWeight'
# df_table.to_sql(sqlite_table, conn, if_exists='append', index=False)

#maxactivity
df_table = df_blend_target
sqlite_table = 'MaxActivity'
df_table.to_sql(sqlite_table, conn, if_exists='append', index=False)

# #MinGenGroupTarget
# df_table = df_target
# sqlite_table = 'MinGenGroupTarget'
# df_table.to_sql(sqlite_table, conn, if_exists='append', index=False)

#MinActivity
df_table = df_target
sqlite_table = 'MinActivity'
df_table.to_sql(sqlite_table, conn, if_exists='append', index=False)

#TechInputSplitAverage
if single_region==0:
    df_table = df_rps
    sqlite_table = 'TechInputSplitAverage'
    df_table.to_sql(sqlite_table, conn, if_exists='append', index=False)
    
#StorageDuration
df_table = df_stordur
sqlite_table = 'StorageDuration'
df_table.to_sql(sqlite_table, conn, if_exists='append', index=False)

# #groups
# df_table = pd.DataFrame(columns=['group_name', 'notes'])
# df_table.loc[:,'group_name'] = df_weights['group_name'].unique()
# sqlite_table = 'groups'
# df_table.to_sql(sqlite_table, conn, if_exists='append', index=False)

# #tech_groups
# df_table = pd.DataFrame(columns=['tech', 'notes'])
# df_table.loc[:,'tech'] = df_weights['tech'].unique()
# df_table['tech'] = df_table['tech'].str.replace('E_COALSTM_N_', 'E_COALSTM_R_')
# sqlite_table = 'tech_groups'
# df_table.to_sql(sqlite_table, conn, if_exists='append', index=False)

if single_region==0 & ('E_TRANS_R' in df_efficiency.tech.unique()):
    df_table = pd.DataFrame(columns = ['tech', 'notes'])
    df_table.loc[0] = ['E_TRANS_R','#existing electric transmission']
    df_table.loc[1] = ['E_TRANS_N','#new electric transmission']
    if include_co2_trans==1:
        df_table.loc[2] = ['co2_TRANS_N','new inter-region co2 transport pipelines']
    sqlite_table = 'tech_exchange'
    df_table.to_sql(sqlite_table, conn, if_exists='append', index=False)

#write CO2, nox_elc and so2_elc emissions for CCS technologies to EmissionActivity table
df_table = pd.concat([df_new_emiss, ccs_techs_emissions])
df_table = df_table[df_emiss.columns]
df_table['tech'] = df_table['tech'].str.replace('E_COALSTM_N_', 'E_COALSTM_R_')
sqlite_table = 'EmissionActivity'
df_table.to_sql(sqlite_table, conn, if_exists='append', index=False)

# #write techoutputsplit
# df_table = df_tosplit_ccs
# df_table['tech'] = df_table['tech'].str.replace('E_COALSTM_N_', 'E_COALSTM_R_')
# sqlite_table = 'TechOutputSplit'
# write_sql(df_table, sqlite_table, outputdB)

#new table tech_new_cluster

c.execute("CREATE TABLE IF NOT EXISTS tech_new_cluster ( regions TEXT, tech TEXT, cap_size REAL, PRIMARY KEY (regions, tech))")

sqlite_table = 'tech_new_cluster'
df_table = df_technology_new_cluster.drop_duplicates()
df_table['tech'] = df_table['tech'].str.replace('E_COALSTM_N_', 'E_COALSTM_R_')
df_table.to_sql(sqlite_table, conn, if_exists='append', index=False)

#update linkedtechs table to delete techs not in df_technologies
df_table = df_linkedtechs
sqlite_table = 'LinkedTechs'
df_table.to_sql(sqlite_table, conn, if_exists='append', index=False)

#remove any pb (baseload) flags
if timeslice!=1:
    query = "UPDATE technologies SET flag='p' WHERE flag='pb'"
    c.execute(query)

conn.commit()
conn.close()

In [ ]:
#map regions to descriptions
if single_region!=1:
    map_region_desc = dict()
    map_region_desc['CA'] = 'California'
    map_region_desc['NW'] = 'Northwestern US'
    map_region_desc['SW'] =  'Southwestern US'
    map_region_desc['TX'] =  'Texas'
    map_region_desc['N_CEN'] =  'North Central US'
    map_region_desc['CEN'] =  'Central US'
    map_region_desc['SE'] =  'Southeastern US'
    map_region_desc['MID_AT'] =  'Mid Atlantic US'
    map_region_desc['NE'] = 'Northeastern US'
else:
    map_region_desc = dict()
    map_region_desc['US'] = 'United States'

df_regions = pd.DataFrame.from_dict(map_region_desc, orient='index').reset_index()
df_regions.columns = ['regions', 'region_note']

In [ ]:
conn = sqlite3.connect(outputdB)
c = conn.cursor()

if remove_elec==1:
    query = "DELETE FROM Efficiency WHERE output_comm='ELCP' OR output_comm LIKE 'ELCP_Renewables%' \
    OR output_comm='ELCDIST_R' OR tech LIKE 'E_Batt%' OR tech LIKE 'E_TRANS%'"
    c.execute(query)
    #query = "DELETE FROM Efficiency WHERE input_comm='ELCP' OR input_comm='ELCP_Renewables'"
    #c.execute(query)
    query = "DELETE FROM tech_ramping"
    c.execute(query)
    query = "DELETE FROM RampUp"
    c.execute(query)
    query = "DELETE FROM RampDown"
    c.execute(query)
    query = "DELETE FROM MinGenGroupTarget"
    c.execute(query)

if elec_only==1:
    elec_demand = pd.read_csv(file_prefix + 'elec_demand.csv')
    demands = elec_demand.groupby('periods').sum().reset_index()
    demands = demands.melt(id_vars='periods')
    demands.rename(columns={'region':'regions', 'value':'demand', 'variable':'regions'}, inplace=True)
    demands['periods'] = demands['periods'].astype(int)
    demands.insert(2,'demand_comm', 'ELC_dem')
    demands.loc[:,'demand'] *=3.6e-6 #convert MWh to PJ
    demands.loc[:,'demand_units'] = 'PJ'
    demands.loc[:,'demand_notes'] = 'from PowerGenome'
    df_table = demands[['regions','periods','demand_comm','demand','demand_units','demand_notes']]
    df_table.to_sql('Demand',conn, if_exists='append', index=False)
    
    df_elc_comm = pd.DataFrame(columns=['comm_name', 'flag','comm_desc'], data=[['ELC_dem','d','Final electricity load']])
    df_elc_comm.to_sql('commodities',conn, if_exists='append', index=False)
    
    df_elc_tech = pd.DataFrame(columns=['tech', 'flag','sector','tech_desc','tech_category'], data=[['ELC_BLND','p', 'electric','dummy technology to meet electricity demand commodity', '']])
    df_elc_tech.to_sql('technologies',conn, if_exists='append', index=False)
    
    df_elc_tech = pd.DataFrame(columns=['regions', 'input_comm', 'tech', 'vintage','output_comm', 'efficiency', 'eff_notes'])
    df_elc_tech.loc[:,'regions'] = df_regions['regions'].unique()
    df_elc_tech.loc[:,'input_comm'] = 'ELC'
    df_elc_tech.loc[:,'tech'] = 'ELC_BLND'
    df_elc_tech.loc[:,'output_comm'] = 'ELC_dem'
    df_elc_tech.loc[:,'efficiency'] = 1
    df_elc_tech.loc[:,'vintage'] = start_year
    df_elc_tech.loc[:,'eff_notes'] = 'dummy technology to meet electricity demand commodity'
    df_elc_tech.to_sql('Efficiency',conn, if_exists='append', index=False)
    
    query = "UPDATE Efficiency SET output_comm='ELCP_Renewables' WHERE tech LIKE 'E_SOLPVENDUSE%'"
    c.execute(query)
    query = "UPDATE MinActivity set regions='global' WHERE regions='US'"
    c.execute(query)
    query = "UPDATE MaxActivity set regions='global' WHERE regions='US'"
    c.execute(query)
    
    for table in table_list:
        df_cols = c.execute("SELECT * FROM pragma_table_info('" + table[0] + "')").fetchall()
        df_cols = [x[1] for x in df_cols]

        if table[0]=='regions':
            query = "DELETE FROM " + table[0]
            c.execute(query)
            model_regions_df = df_regions[df_regions['regions'].isin(model_regions)].reset_index(drop=True)
            model_regions_df.to_sql(table[0],conn, if_exists='append', index=False)
        elif 'regions' in df_cols:
            df_remains = pd.read_sql_query("SELECT * FROM " + table[0] + " WHERE regions='US'", conn)
#             query = "DELETE FROM " + table[0] + " WHERE regions='US'"
            query = "DELETE FROM " + table[0] + " WHERE regions NOT IN ('" + "','".join(model_regions) + "')"
            c.execute(query)
            for reg in model_regions:
                df_remains_new = df_remains.copy()
                df_remains_new.regions=reg
                if table[0] in ['ExistingCapacity']:
                    df_remains_new['exist_cap'] /=9
#                 if table[0] in ['MinActivity']:
#                     df_remains_new['minact'] /=20
                try:
                    df_remains_new.to_sql(table[0],conn, if_exists='append', index=False)
                except:
                    pass
        elif 'primary_region' in df_cols:
            df_remains = pd.read_sql_query("SELECT * FROM " + table[0] + " WHERE primary_region='US'", conn)
            query = "DELETE FROM " + table[0] + " WHERE primary_region='US'"
            c.execute(query)
            for reg in model_regions:
                df_remains_new = df_remains.copy()
                df_remains_new['primary_region']=reg
                try:
                    df_remains_new.to_sql(table[0],conn, if_exists='append', index=False)
                except:
                    pass

iterval = 0
while len(c.execute("SELECT * FROM Efficiency WHERE output_comm NOT IN (SELECT input_comm FROM Efficiency)\
                       AND output_comm NOT IN (SELECT comm_name FROM commodities WHERE flag='d');").fetchall()) > 0:

    c.execute("DELETE FROM Efficiency WHERE output_comm NOT IN (SELECT input_comm FROM Efficiency) \
             AND output_comm NOT IN (SELECT comm_name FROM commodities WHERE flag='d');")
    iterval+=1
    if iterval>10:
        break
print(iterval)

for table in table_list:
    if table[0] != 'Efficiency':  
        df_cols = c.execute("SELECT * FROM pragma_table_info('" + table[0] + "')").fetchall()
        df_cols = [x[1] for x in df_cols]
        if 'tech' in df_cols:
            c.execute("UPDATE "+str(table[0])+" SET tech = TRIM(tech, CHAR(37,13,10));")
            try:
                if 'vintage' in df_cols:
                    # If t doesn't exist in Efficiency table after the deletions made above, 
                    # it is deleted from other tables.
                    c.execute("DELETE FROM "+str(table[0])+" WHERE tech || vintage \
                        NOT IN (SELECT tech || vintage FROM Efficiency)")
                else:
                    c.execute("DELETE FROM "+str(table[0])+" WHERE tech NOT IN (SELECT tech FROM Efficiency);")
            except:
                print(table[0])
        if table[0] == 'EmissionActivity':
            c.execute("DELETE FROM EmissionActivity WHERE input_comm || tech || vintage || output_comm \
                        NOT IN (SELECT input_comm || tech || vintage || output_comm FROM Efficiency)")


c.execute("UPDATE commodities SET comm_name = TRIM(comm_name, CHAR(10,13,37))")
    # delete unused commodities otherwise the model throws an error
c.execute("DELETE FROM commodities WHERE flag!='e' AND comm_name NOT IN (SELECT input_comm from Efficiency UNION SELECT output_comm from Efficiency);")

#update buildings sector
if elec_only!=1:
    #delete unused input_comm/technology rows from the efficiency and techinput split table
    df_buildings_eff_del = pd.read_csv('buildings/TablesForDB/efficiency_buildings_delete.csv')
    for ind, row in df_buildings_eff_del.iterrows():
        query = "DELETE FROM Efficiency WHERE tech = '" + row['tech'] + "' AND input_comm = '" +\
        row['input_comm'] + "'"
        c.execute(query)
        query = "DELETE FROM techinputsplit WHERE tech = '" + row['tech'] + "' AND input_comm = '" +\
        row['input_comm'] + "'"
        c.execute(query)

    #update regional building demands 
    df_buildings = pd.read_csv('buildings/TablesForDB/annualDemand_buildings_regional.csv')
    df_buildings = df_buildings[['regions', 'periods', 'demand_comm', 'demand','demand_units']]
    df_buildings['demand_notes'] = ''
    new_dems = []#['RCW', 'RCD']#, 'RFZ', 'ROTHER']'RCK', , 'RDW', 
    df_buildings = df_buildings[~df_buildings['demand_comm'].isin(new_dems)]
#     df_buildings.loc[df_buildings['demand_comm']=='RCD','demand'] = 0
#     df_buildings.loc[df_buildings['demand_comm']=='RDW','demand'] = 0
#     df_buildings.loc[df_buildings['demand_comm']=='RCK','demand'] = 0
#     df_buildings.loc[df_buildings['demand_comm']=='RCW','demand'] = 0
#     df_buildings.loc[df_buildings['demand_comm']=='RFZ','demand'] = 0
#     df_buildings.loc[df_buildings['demand_comm']=='ROTHER','demand'] = 0
    df_buildings = df_buildings[df_buildings['periods'].isin(all_periods)]
    if single_region==1:
        df_buildings = df_buildings.groupby(by=['periods', 'demand_comm','demand_units','demand_notes']).sum().reset_index()
        df_buildings.insert(0,'regions','US')
        df_buildings = df_buildings[['regions', 'periods', 'demand_comm', 'demand','demand_units','demand_notes']]
    old_demands = pd.read_sql_query("SELECT DISTINCT(demand_comm) FROM Demand WHERE substr(demand_comm,1,1) = 'C'OR substr(demand_comm,1,1) = 'R'", conn)
    query = "DELETE FROM Demand WHERE \
    substr(demand_comm,1,1) = 'C'OR substr(demand_comm,1,1) = 'R'"
    c.execute(query)
    df_buildings.drop_duplicates(subset=['regions','periods','demand_comm'], inplace=True)
    df_buildings.to_sql('Demand',conn, if_exists='append', index=False) 
    
    delete_demands = set([x[0] for x in old_demands.values]).difference(set(df_buildings['demand_comm'].unique()))
    query = 'DELETE FROM DemandSpecificDistribution WHERE demand_name IN ("' + '", "'.join(delete_demands) + '")'
    c.execute(query)
    
    #add new demands to commodities table
    query = 'SELECT comm_name FROM commodities WHERE flag="d"'
    df_exist_demand = pd.read_sql_query(query, conn)
    df_demand_comm = pd.DataFrame(columns=['comm_name','flag','comm_desc'])
    df_demand_comm.loc[:,'comm_name'] = list(set(df_buildings.loc[:,'demand_comm'].unique()) - set(df_exist_demand['comm_name']))
    df_demand_comm.loc[:,'flag'] = 'd'
    df_demand_comm.loc[:,'comm_desc'] = 'residential demand'
    df_demand_comm.to_sql('commodities',conn, if_exists='append', index=False)
    
    #update regional building efficiencies
    df_buildings_eff_all = pd.read_csv('buildings/TablesForDB/efficiency_buildings_regional.csv')
    df_buildings_eff = df_buildings_eff_all[['regions', 'input_comm','tech','vintage','output_comm', 'efficiency', 'eff_notes']]
    df_buildings_eff = df_buildings_eff[~df_buildings_eff['tech'].str.contains('R_LT_LED60HE_GSL_N')]
    df_buildings_eff = df_buildings_eff[~df_buildings_eff['tech'].str.contains('R_LT_LED60ST_GSL_N')]
    df_buildings_eff.loc[df_buildings_eff['efficiency']=='#VALUE!','efficiency'] = 1
    df_buildings_eff['efficiency'] = pd.to_numeric(df_buildings_eff['efficiency'])
    df_buildings_eff['tech'] = df_buildings_eff['tech'].str.strip()
    df_buildings_eff.drop_duplicates(subset=['regions', 'input_comm','tech','vintage','output_comm'],inplace=True)
    if single_region==1:
        df_buildings_eff = df_buildings_eff.groupby(by=['input_comm','tech','vintage','output_comm', 'eff_notes']).mean().reset_index()
        df_buildings_eff.insert(0,'regions','US')
        df_buildings_eff = df_buildings_eff[['regions', 'input_comm','tech','vintage','output_comm', 'efficiency', 'eff_notes']]
    query = "DELETE FROM Efficiency WHERE \
    (substr(output_comm,1,1) = 'C'OR substr(output_comm,1,1) = 'R') AND \
    (output_comm IN (SELECT DISTINCT(comm_name) FROM commodities WHERE flag='d')) AND (tech NOT LIKE '%BLND%')"
    c.execute(query)
    query = 'DELETE FROM Efficiency WHERE tech IN ("' + '", "'.join(df_buildings_eff['tech'].unique()) + '")'
    c.execute(query)
    query = 'DELETE FROM Efficiency WHERE output_comm IN ("' + '", "'.join(delete_demands) + '")'
    c.execute(query)
    query = 'DELETE FROM Efficiency WHERE output_comm IN ("' + '", "'.join(df_buildings_eff['output_comm'].unique()) + '") \
    AND output_comm NOT IN (SELECT DISTINCT(demand_comm) FROM Demand)'
    c.execute(query)
    df_buildings_eff = df_buildings_eff[~df_buildings_eff['output_comm'].isin(new_dems)] #FIX
    df_buildings_eff.to_sql('Efficiency',conn, if_exists='append', index=False)
            
    iterval = 0
    while len(c.execute("SELECT * FROM Efficiency WHERE output_comm NOT IN (SELECT input_comm FROM Efficiency)\
                           AND output_comm NOT IN (SELECT comm_name FROM commodities WHERE flag='d');").fetchall()) > 0:

        c.execute("DELETE FROM Efficiency WHERE output_comm NOT IN (SELECT input_comm FROM Efficiency) \
                 AND output_comm NOT IN (SELECT comm_name FROM commodities WHERE flag='d');")
        iterval+=1
        if iterval>10:
            break
    print(iterval)
    
    #add techinputsplit
    query = 'DELETE FROM TechInputSplit WHERE tech NOT IN (SELECT DISTINCT(tech) FROM Efficiency)'
    c.execute(query)
    #remove cooking service demand splits that constrain NG use for cooking in US_BASE
    query = 'DELETE FROM TechInputSplit WHERE tech="C_BLND_FUEL_CK"'
    c.execute(query)
    #remove cooling service demand splits that constrain cooling type in US_BASE
    query = 'DELETE FROM TechInputSplit WHERE tech="C_BLND_FUEL_SC"'
    c.execute(query)
    #remove cooling service demand splits that constrain cooling type in US_BASE
    query = 'DELETE FROM TechInputSplit WHERE tech="R_BLND_FUEL_SC"'
    c.execute(query)
    
    df_buildings_tisplit = pd.read_csv('buildings/TablesForDB/tech_input_split.csv', index_col=0)
    df_buildings_tisplit['ti_split_notes'].fillna('', inplace=True)
    df_buildings_tisplit = df_buildings_tisplit[df_buildings_tisplit['periods']>=2020]
    existing_techs_tisplit = df_buildings_tisplit['tech'].unique()
    
    query = 'DELETE FROM TechInputSplit WHERE tech IN ("' + '", "'.join(existing_techs_tisplit) + '")'
    c.execute(query)

    if single_region==1:
        df_buildings_tisplit = df_buildings_tisplit[df_buildings_tisplit['regions']=='US']
    else:
        df_buildings_tisplit = df_buildings_tisplit[df_buildings_tisplit['regions']!='US']
#         df_buildings_tisplit = df_buildings_tisplit.groupby(by=['periods','input_comm','tech', 'ti_split_notes']).mean().reset_index()
#         df_buildings_tisplit.insert(0,'regions','US')
#         df_buildings_tisplit = df_buildings_tisplit[['regions', 'periods','input_comm','tech', 'ti_split','ti_split_notes']]
        
    df_buildings_tisplit.to_sql('TechInputSplit',conn, if_exists='append', index=False)
    
    
    #update buildings costs invest
    df_buildings_invest_all = pd.read_csv('buildings/TablesForDB/costInvest_buildings_regional.csv')
#     df_buildings_invest_all['cost_invest'] *= 10**6
#     df_buildings_invest_all = df_buildings_invest_all[~df_buildings_invest_all['cost_invest'].isna()]
    df_buildings_invest_all = df_buildings_invest_all[~df_buildings_invest_all['tech'].str.contains('R_LT_LED60HE_GSL_N')]
    df_buildings_invest_all = df_buildings_invest_all[~df_buildings_invest_all['tech'].str.contains('R_LT_LED60ST_GSL_N')]
    df_buildings_invest = df_buildings_invest_all[['regions','tech','vintage','cost_invest','cost_invest_units']]
    df_buildings_invest['cost_invest_notes'] = df_buildings_invest_all['capacity'].map(str) + ' ' + df_buildings_invest_all['capacity units']
    df_buildings_invest.drop_duplicates(subset=['regions', 'tech','vintage'], inplace=True)
    if single_region==1:
        df_buildings_invest = df_buildings_invest.groupby(by=['tech','vintage', 'cost_invest_units']).mean().reset_index()
        df_buildings_invest.insert(0,'regions','US')
        df_buildings_invest = df_buildings_invest[['regions','tech','vintage','cost_invest','cost_invest_units']]
    query = 'DELETE FROM CostInvest WHERE tech IN ("' + '", "'.join(df_buildings_invest['tech'].unique()) + '")'
    c.execute(query)
    query = 'DELETE FROM CostInvest WHERE tech NOT IN (SELECT DISTINCT(tech) FROM Efficiency)'
    c.execute(query)
    df_buildings_invest.to_sql('CostInvest',conn, if_exists='append', index=False)
    query = 'DELETE FROM CostInvest WHERE tech NOT IN (SELECT DISTINCT(tech) FROM Efficiency)' #FIX
    c.execute(query) #FIX
   
    #update buildings costs fixed
    df_buildings_fixed_all = pd.read_csv('buildings/TablesForDB/costFixed_buildings_regional.csv')
    df_buildings_fixed_all = df_buildings_fixed_all[~df_buildings_fixed_all['tech'].str.contains('R_LT_LED60HE_GSL_N')]
    df_buildings_fixed_all = df_buildings_fixed_all[~df_buildings_fixed_all['tech'].str.contains('R_LT_LED60ST_GSL_N')]
    df_buildings_fixed = df_buildings_fixed_all[['regions','periods','tech','vintage','cost_fixed','cost_fixed_units']]
    df_buildings_fixed['cost_fixed_notes'] = df_buildings_fixed_all['capacity'].map(str) + ' ' + df_buildings_fixed_all['capacity_units']
    df_buildings_fixed.drop_duplicates(subset=['regions', 'periods','tech','vintage'], inplace=True)
    df_buildings_fixed = df_buildings_fixed[df_buildings_fixed['periods'].isin(all_periods)]
    if single_region==1:
        df_buildings_fixed = df_buildings_fixed.groupby(by=['periods','tech','vintage','cost_fixed_units']).mean().reset_index()
        df_buildings_fixed.insert(0,'regions','US')
        df_buildings_fixed = df_buildings_fixed[['regions','periods','tech','vintage','cost_fixed','cost_fixed_units']]
    query = 'DELETE FROM Costfixed WHERE tech IN ("' + '", "'.join(df_buildings_fixed['tech'].unique()) + '")'
    c.execute(query)
    df_buildings_fixed.to_sql('CostFixed',conn, if_exists='append', index=False)
    query = 'DELETE FROM CostFixed WHERE tech NOT IN (SELECT DISTINCT(tech) FROM Efficiency)' #FIX
    c.execute(query)
    
    #update variable costs for techs no longer needed for the buildings sector
    query = 'DELETE FROM CostVariable WHERE tech NOT IN (SELECT DISTINCT(tech) FROM Efficiency)'
    c.execute(query)
    
    #update EmissionActivity for techs no longer needed for the buildings sector
    query = 'DELETE FROM EmissionActivity WHERE tech NOT IN (SELECT DISTINCT(tech) FROM Efficiency)'
    c.execute(query)
    
    #update buildings lifetimes
    df_buildings_lifetime = pd.read_csv('buildings/TablesForDB/lifetime_buildings_regional.csv')
    df_buildings_lifetime = df_buildings_lifetime[~df_buildings_lifetime['tech'].str.contains('R_LT_LED60HE_GSL_N')]
    df_buildings_lifetime = df_buildings_lifetime[~df_buildings_lifetime['tech'].str.contains('R_LT_LED60ST_GSL_N')]
    df_buildings_lifetime = df_buildings_lifetime[['regions','tech','life', 'life_notes']]
    df_buildings_lifetime.drop_duplicates(subset=['regions', 'tech'], inplace=True)
    if single_region==1:
        df_buildings_lifetime = df_buildings_lifetime.groupby(by=['tech','life_notes']).mean().reset_index()
        df_buildings_lifetime.insert(0,'regions','US')
        df_buildings_lifetime = df_buildings_lifetime[['regions','tech','life', 'life_notes']]
    query = 'DELETE FROM LifetimeTech WHERE tech IN ("' + '", "'.join(df_buildings_lifetime['tech'].unique()) + '")'
    c.execute(query)
    query = 'DELETE FROM LifetimeTech WHERE tech NOT IN (SELECT DISTINCT(tech) FROM Efficiency)'
    c.execute(query)
    df_buildings_lifetime.to_sql('LifetimeTech',conn, if_exists='append', index=False)
    query = 'DELETE FROM LifetimeTech WHERE tech NOT IN (SELECT DISTINCT(tech) FROM Efficiency)' #FIX
    c.execute(query) #FIX
    
    #update buildings existing capacity
    df_buildings_existcap = pd.concat([pd.read_csv('buildings/TablesForDB/residential_existingCapacity.csv'), \
                        pd.read_csv('buildings/TablesForDB/commercial_existingCapacity.csv')]) 
    df_buildings_existcap.rename(columns={'Region_OEO':'regions', 'ExistingCap':'exist_cap'}, inplace=True)
    df_buildings_existcap = df_buildings_existcap[['regions','tech','vintage','exist_cap']]
    df_buildings_existcap['exist_cap_units'] = ''
    df_buildings_existcap['exist_cap_notes'] = ''
    if single_region==1:
        df_buildings_existcap = df_buildings_existcap.groupby(by=['tech','vintage', 'exist_cap_units', 'exist_cap_notes']).sum().reset_index()
        df_buildings_existcap.insert(0,'regions','US')
        df_buildings_existcap = df_buildings_existcap[['regions','tech','vintage','exist_cap', 'exist_cap_units', 'exist_cap_notes']]
    query = 'DELETE FROM ExistingCapacity WHERE tech IN ("' + '", "'.join(df_buildings_existcap['tech'].unique()) + '")'
    c.execute(query)
    query = 'DELETE FROM ExistingCapacity WHERE tech NOT IN (SELECT DISTINCT(tech) FROM Efficiency)'
    c.execute(query)
    df_buildings_existcap.to_sql('ExistingCapacity',conn, if_exists='append', index=False)
    query = 'DELETE FROM ExistingCapacity WHERE tech NOT IN (SELECT DISTINCT(tech) FROM Efficiency)'#FIX
    c.execute(query) #FIX
    
    #add discount rate
    df_buildings_discrate = pd.read_csv('buildings/TablesForDB/discount_rate.csv')   
    df_buildings_discrate = df_buildings_discrate[~df_buildings_discrate['tech'].str.contains('R_LT_LED60HE_GSL_N')]
    df_buildings_discrate = df_buildings_discrate[~df_buildings_discrate['tech'].str.contains('R_LT_LED60ST_GSL_N')]
    df_buildings_discrate = df_buildings_discrate[['regions','tech','vintage','tech_rate', 'tech_rate_notes']]
    df_buildings_discrate['tech_rate_notes'].fillna('N/A',inplace=True)

    if single_region==1:
        df_buildings_discrate = df_buildings_discrate.groupby(by=['tech','vintage', 'tech_rate_notes']).mean().reset_index()
        df_buildings_discrate.insert(0,'regions','US')
        df_buildings_discrate = df_buildings_discrate[['regions','tech','vintage','tech_rate', 'tech_rate_notes']]
    query = 'DELETE FROM DiscountRate WHERE tech IN ("' + '", "'.join(df_buildings_discrate['tech'].unique()) + '")'
    c.execute(query)
    query = 'DELETE FROM DiscountRate WHERE tech NOT IN (SELECT DISTINCT(tech) FROM Efficiency)'
    c.execute(query)
    df_buildings_discrate.to_sql('DiscountRate',conn, if_exists='append', index=False)  
    query = 'DELETE FROM DiscountRate WHERE tech NOT IN (SELECT DISTINCT(tech) FROM Efficiency)' #FIX
    c.execute(query)#FIX
    
    #update technologies table
    query = 'DELETE FROM technologies WHERE tech NOT in (SELECT DISTINCT(tech) FROM Efficiency)'
    c.execute(query)
    query = 'SELECT DISTINCT(tech) FROM Efficiency WHERE tech NOT IN (SELECT tech FROM technologies)'
    tech_table = pd.read_sql_query(query, conn)
    if len(tech_table)>0:
        tech_table.loc[:, 'flag'] = 'p'
        tech_table.loc[:, 'sector'] = 'residential'
        mask = tech_table['tech'].str[0]=='C'
        tech_table.loc[mask, 'sector'] = 'commercial'
        tech_table.loc[:, 'tech_desc'] = ''
        tech_table.loc[:, 'tech_category'] = ''
        #add tech descriptions
        tech_desc = df_buildings_eff_all[['tech', 'Tech Description']].drop_duplicates()
        #drop duplicates for heat pump techs that have the same description, but different heating or cooling suffix 
        tech_desc['Tech Description'] = [x.split(' – ')[0].strip() if 'HEAT PUMP' in x else x for x in tech_desc['Tech Description'] ]
        tech_desc['Tech Description'] = [x.split(' -')[0].strip() if 'HEAT PUMP' in x else x for x in tech_desc['Tech Description'] ]
        tech_desc['Tech Description'] = [x.split(' FOR SPACE')[0].strip() if 'HEAT PUMP' in x else x for x in tech_desc['Tech Description'] ]
        tech_desc = tech_desc[['tech', 'Tech Description']].drop_duplicates()
        tech_desc['tech_desc'] = tech_desc['Tech Description'].str.lower()
        tech_table = tech_table.drop(columns='tech_desc').merge(tech_desc.drop(columns='Tech Description'), on = 'tech', how='left')
        tech_table = tech_table[['tech', 'flag','sector','tech_desc','tech_category']]
        tech_table.to_sql('technologies',conn, if_exists='append', index=False)
        
    #update commodities table
    query = 'DELETE FROM commodities WHERE comm_name IN ("' + '", "'.join(delete_demands) + '")'
    c.execute(query)
    query = 'DELETE FROM commodities WHERE comm_name NOT in (SELECT DISTINCT(input_comm) FROM Efficiency)\
    AND comm_name NOT in (SELECT DISTINCT(output_comm) FROM Efficiency) AND (flag="p" OR flag="d") '
    c.execute(query)
    query = 'SELECT DISTINCT(input_comm) FROM Efficiency WHERE input_comm NOT IN (SELECT comm_name FROM commodities)'
    tech_table = pd.read_sql_query(query, conn)
    if len(tech_table)>0:
        tech_table.loc[:, 'flag'] = 'p'
        tech_table.loc[:, 'comm_desc'] = ''
        tech_table.rename(columns={'input_comm':'comm_name'}, inplace=True)
        tech_table.to_sql('commodities',conn, if_exists='append', index=False)
    query = 'SELECT DISTINCT(output_comm) FROM Efficiency WHERE output_comm NOT IN (SELECT comm_name FROM commodities)'
    tech_table = pd.read_sql_query(query, conn)
    if len(tech_table)>0:
        tech_table.loc[:, 'flag'] = 'p'
        tech_table.loc[:, 'comm_desc'] = ''
        tech_table.rename(columns={'output_comm':'comm_name'}, inplace=True)
        tech_table.to_sql('commodities',conn, if_exists='append', index=False) 

if elec_only!=1 :
# if (single_region!=1) & (elec_only!=1) :
    #update regional transportation demands 
    transp_file = 'transport/transport_regionalization_master.xlsx'
    transp_sheets = pd.ExcelFile(transp_file)
    df_trans_del = pd.read_excel(transp_file, sheet_name='needs deleting', header=None)
    for sheet in transp_sheets.sheet_names:
        if sheet!='needs deleting':
            df_trans = pd.read_excel(transp_file, sheet_name=sheet)
            if sheet=='ExistingCapacity':
                df_trans = df_trans[df_trans['exist_cap']>0]
                df_trans['exist_cap'] *= 2/3 #adjust so that only 2/3 capacity survives to 2020
            if sheet=='Demand':
                query = 'DELETE FROM Demand WHERE demand_comm IN ("' + '", "'.join(df_trans['demand_comm'].unique()) + '")'
                c.execute(query)
                df_trans.to_sql(sheet,conn, if_exists='append', index=False)
#             elif 'vintage' in df_trans.columns: 
#                 df_trans['query'] = 'tech = "' + df_trans['tech'] + '" AND vintage = ' + df_trans['vintage'].map(str)
#                 query = 'DELETE FROM ' + sheet + ' WHERE (' + ') OR ('.join(df_trans['query']) + ')'
#                 if sheet=='Efficiency':
#                     print(query)
#                     break
#                 c.execute(query)
#                 df_trans.drop(columns='query').to_sql(sheet,conn, if_exists='append', index=False)
#             elif sheet=='TechInputSplit':
#                 query = 'DELETE FROM ' + sheet + ' WHERE tech IN ("' + '", "'.join(df_trans['tech'].unique()) + '")'
#                 c.execute(query)
#                 df_trans.to_sql(sheet,conn, if_exists='append', index=False)
#             elif 'regions' in df_trans.columns:
#                 df_trans.to_sql(sheet,conn, if_exists='append', index=False)
            else:
                query = 'DELETE FROM ' + sheet + ' WHERE tech IN ("' + '", "'.join(df_trans['tech'].unique()) + '")'
                c.execute(query)

                df_trans.to_sql(sheet,conn, if_exists='append', index=False)

    for table in table_list:
        df_cols = c.execute("SELECT * FROM pragma_table_info('" + table[0] + "')").fetchall()
        df_cols = [x[1] for x in df_cols]
        if 'tech' in df_cols:
            query = 'DELETE FROM ' + table[0] + ' WHERE tech IN ("' + '", "'.join(df_trans_del[0].unique()) + '")'
            c.execute(query)
    
    #add fuels transport
    transport_fuels = ['ETH_CORN', 'ETH_CEL']#['GAS', 'E10']
    df_transp_fuels = pd.DataFrame(columns=['regions','input_comm','tech','vintage','output_comm','efficiency', 'eff_notes'])
    df_transp_fuels_regs = []
    for i in combinations(regions_list,2):
        df_transp_fuels_regs.append('-'.join(i))
    df_transp_fuels['regions'] = df_transp_fuels_regs
    df_transp_fuels['vintage'] = start_year
    df_transp_fuels['efficiency'] = 0.99
    df_transp_fuels_swap = df_transp_fuels.copy()
    df_transp_fuels_swap['regions'] = df_transp_fuels['regions'].apply(swap_regions)
    df_transp_fuels = pd.concat([df_transp_fuels, df_transp_fuels_swap])
    for fu in transport_fuels:
        df_transp_fuels['input_comm'] = fu
        df_transp_fuels['output_comm'] = fu
        df_transp_fuels['tech'] = fu + '_TRANS'
        df_transp_fuels.to_sql('Efficiency',conn, if_exists='append', index=False)
        tech_exchange = [fu + '_TRANS', fu + ' inter-region transport']
        tech_exchange = pd.DataFrame(data=[tech_exchange], columns=['tech', 'notes'])
        tech_exchange.to_sql('tech_exchange',conn, if_exists='append', index=False)
#         if fu!='GAS':
#             tech_exchange.to_sql('tech_annual',conn, if_exists='append', index=False)
        techn = [fu + '_TRANS', 'p', 'transport', fu + ' inter-region transport', '']
        techn = pd.DataFrame(data=[techn], columns=['tech', 'flag', 'sector', 'tech_desc', 'tech_category'])
        techn.to_sql('technologies',conn, if_exists='append', index=False)

    #update regional industrial sector
#     df_industry = pd.read_csv('industry/industry_regional_demands_adjusted.csv')
#     df_industry = df_industry[['regions', 'periods', 'demand_comm', 'demand','demand_units','demand_notes']]
#     query = "DELETE FROM Demand  \
#     WHERE demand_comm LIKE '%IND%'"
#     c.execute(query)
#     df_industry.to_sql('Demand',conn, if_exists='append', index=False)
    cwd = os.getcwd()
    os.chdir('industry')
    %run industry_integration_OEO.ipynb
    os.chdir(cwd)

    #find remaining demands at US level, divide by 9 to get demands in each region
    df_remains = pd.read_sql_query("SELECT * FROM Demand WHERE regions='US'", conn)
    query = "DELETE FROM Demand WHERE regions='US'"
    c.execute(query)
    for reg in model_regions:
        df_remains_new = df_remains.copy()
        df_remains_new.regions=reg
        df_remains_new['demand'] /=9
        df_remains_new.to_sql('Demand',conn, if_exists='append', index=False)
        
    query = "UPDATE MinActivity set regions='global' WHERE regions='US'"
    c.execute(query)
    query = "UPDATE MaxActivity set regions='global' WHERE regions='US'"
    c.execute(query)
    
    for table in table_list:
        df_cols = c.execute("SELECT * FROM pragma_table_info('" + table[0] + "')").fetchall()
        df_cols = [x[1] for x in df_cols]

        if table[0]=='regions':
            query = "DELETE FROM " + table[0]
            c.execute(query)
            df_regions.to_sql(table[0],conn, if_exists='append', index=False)
        elif 'regions' in df_cols:
            df_remains = pd.read_sql_query("SELECT * FROM " + table[0] + " WHERE regions='US'", conn)
            query = "DELETE FROM " + table[0] + " WHERE regions='US'"
            c.execute(query)
            for reg in model_regions:
                df_remains_new = df_remains.copy()
                df_remains_new.regions=reg
                if table[0] in ['ExistingCapacity']:
                    df_remains_new['exist_cap'] /=9
#                 if table[0] in ['MinActivity']:
#                     df_remains_new['minact'] /=20
                if table[0] != 'TechInputSplitAverage': #don't split any TechInputSplitAverage data             
                    df_remains_new.to_sql(table[0],conn, if_exists='append', index=False)
        elif 'primary_region' in df_cols:
            df_remains = pd.read_sql_query("SELECT * FROM " + table[0] + " WHERE primary_region='US'", conn)
            query = "DELETE FROM " + table[0] + " WHERE primary_region='US'"
            c.execute(query)
            for reg in model_regions:
                df_remains_new = df_remains.copy()
                df_remains_new['primary_region']=reg
                df_remains_new.to_sql(table[0],conn, if_exists='append', index=False)
            
    #fix issue where some existing transport techs were deleted in the regional db
    for table in table_list:
        df_cols = c.execute("SELECT * FROM pragma_table_info('" + table[0] + "')").fetchall()
        df_cols = [x[1] for x in df_cols]
        if 'regions' in df_cols and 'tech' in df_cols and 'vintage' in df_cols:
            c.execute("DELETE FROM "+str(table[0])+" WHERE regions || tech || vintage \
                        NOT IN (SELECT regions || tech || vintage FROM ExistingCapacity)\
                        AND vintage < " + str(start_year))

query = "DELETE FROM LinkedTechs WHERE primary_tech NOT IN (SELECT tech FROM technologies)"
c.execute(query)



In [ ]:
#run jupyter notebook that adds EIA planned coal plant retirements
%run electric/Coal_Retirements.ipynb

In [ ]:
#update electric sector so2 and Nox emisions limits
df_emislimits = pd.read_excel('electric/regional_elec_emissions_limits.xlsx', sheet_name = 'Formatted for temoa')
query = 'DELETE FROM EmissionLimit WHERE emis_comm IN ("' + '", "'.join(df_emislimits['emis_comm'].unique()) + '")'
c.execute(query)
df_emislimits.to_sql('EmissionLimit',conn, if_exists='append', index=False)

In [ ]:
# #update coal and ng electric sector emissions factors

query = "DELETE FROM EmissionActivity WHERE tech LIKE 'E_COALSTM%' AND emis_comm LIKE '%_ELC'"
c.execute(query)
query = "DELETE FROM EmissionActivity WHERE emis_comm LIKE '%_ELC' AND tech LIKE 'E_EA_COA%' and output_comm NOT LIKE '%_IGCC_N'"
c.execute(query)
query = "DELETE FROM EmissionActivity WHERE emis_comm LIKE '%_ELC' AND tech LIKE 'E_NGA%' AND vintage < " + str(start_year)
c.execute(query)

df_emisact = pd.read_csv('electric/Regional_EAs.csv')
df_emisact = df_emisact.merge(df_efficiency[['regions','input_comm','tech', 'vintage', 'output_comm']], left_on = ['Region', 'input_com'], right_on = ['regions', 'input_comm'], how='left')
df_emisact = df_emisact[df_emisact['vintage']<start_year]
df_emisact = df_emisact[['regions','emis_comm','input_comm', 'tech', 'vintage', 'output_comm', 'emis_act', 'emis_act_units', 'emis_act_notes']]
df_emisact['tech'] = df_emisact['tech'].str.replace('E_COALSTM_N_', 'E_COALSTM_R_')

df_emisact.to_sql('EmissionActivity', conn, if_exists='append', index=False)

In [ ]:
#add regional co2 sequestration limits, annual and total
df_co2_seq_maxresource = pd.read_excel('supply/co2_sequestration/co2_sequestration_regional.xlsx', sheet_name='MaxResource')
df_co2_seq_maxresource['maxres'] = np.round(df_co2_seq_maxresource['maxres'],0)

query = "DELETE FROM MaxResource WHERE tech in ('" + "','".join(df_co2_seq_maxresource['tech'].unique()) + "')"
c.execute(query)
df_co2_seq_maxresource = df_co2_seq_maxresource[df_co2_seq_maxresource['regions'].isin(model_regions)]
df_co2_seq_maxresource.to_sql('MaxResource', conn, if_exists='append', index=False)

df_co2_seq_maxactivity = pd.read_excel('supply/co2_sequestration/co2_sequestration_regional.xlsx', sheet_name='MaxActivity')
df_co2_seq_maxactivity['maxact'] = np.round(df_co2_seq_maxactivity['maxact'],0)

query = "DELETE FROM MaxActivity WHERE tech in ('" + "','".join(df_co2_seq_maxactivity['tech'].unique()) + "')"
c.execute(query)
df_co2_seq_maxactivity = df_co2_seq_maxactivity[df_co2_seq_maxactivity['regions'].isin(model_regions)]
df_co2_seq_maxactivity.to_sql('MaxActivity', conn, if_exists='append', index=False)



In [ ]:
#add co2_elc commodity to all electric sector technologies (use phase only, not upstream)
query = "SELECT * FROM EmissionActivity WHERE emis_comm='co2' \
AND tech IN (SELECT tech FROM technologies WHERE sector LIKE 'electric%')"
df_co2_elc = pd.read_sql_query(query, conn)
df_co2_elc['emis_comm'] = 'co2_ELC'
df_co2_elc.to_sql('EmissionActivity', conn, if_exists='append', index=False)

df_elc_comm = pd.DataFrame(columns=['comm_name', 'flag','comm_desc'], data=[['co2_ELC','e','CO2 emissions from the electric sector']])
df_elc_comm.to_sql('commodities',conn, if_exists='append', index=False)

In [ ]:
#run jupyter notebook that adds biomass supply curve
cwd = os.getcwd()
os.chdir('supply/biomass/')
%run BiomassSupplyCurves.ipynb
os.chdir(cwd)

if elec_only==1:
    query = "DELETE FROM Efficiency WHERE regions NOT IN ('" + "','".join(model_regions) + "')"
    c.execute(query)
    query = "DELETE FROM Efficiency WHERE output_comm IN ('SOY','CORN','STV')"
    c.execute(query)
    query = "DELETE FROM technologies WHERE tech NOT IN (SELECT DISTINCT tech FROM Efficiency)"
    c.execute(query)
    query = "DELETE FROM MaxActivity WHERE tech NOT IN (SELECT DISTINCT tech FROM Efficiency)"
    c.execute(query)
    query = "DELETE FROM CostVariable WHERE tech NOT IN (SELECT DISTINCT tech FROM Efficiency)"
    c.execute(query)
    query = "DELETE FROM commodities WHERE comm_name IN ('SOY','CORN','STV')"
    c.execute(query)


In [ ]:
#incorporate California cap and trade emissions limits
if (df_regions['regions']=='CA').any() & elec_only==0:
    df_ca_cap = pd.read_excel('policies/CA_capandtrade_approximation.xlsx')
    emissions_cap = df_ca_cap[df_ca_cap['Field']=='2030 energy sector emissions target']['Value']*1000 #ktons
    emissions_cap_df = pd.DataFrame(columns=['regions','periods','emis_comm','emis_limit', 'emis_limit_units','emis_limit_notes'])
    emissions_cap_df['periods'] = np.arange(2030, 2051, 5)
    emissions_cap_df['regions']= 'CA'
    emissions_cap_df['emis_comm']= 'co2'
    emissions_cap_df['emis_limit']= emissions_cap.iloc[0]
    emissions_cap_df['emis_limit_units']= 'kt'
    emissions_cap_df['emis_limit_notes']= '40% below 1990 levels, assuming 86% of CA emissions from energy, ignoring imports'
    emissions_cap_df.to_sql('EmissionLimit',conn, if_exists='append', index=False)


In [ ]:
#normalize FOM to VOM for coal and NGCC assuming 65% capacity factor
# df_FOM = pd.read_sql_query("SELECT * FROM CostFixed WHERE tech LIKE 'E_COALSTM_R%' OR tech LIKE 'E_NGACC_R%'", conn)
# df_VOM = pd.read_sql_query("SELECT * FROM CostVariable WHERE tech LIKE 'E_COALSTM_R%' OR tech LIKE 'E_NGACC_R%'",conn)
# df_FOM['cost_fixed']*=10**6/1000/(8760*0.65)
# df_VOM = df_VOM.merge(df_FOM[['regions','periods','tech','vintage','cost_fixed']])
# df_VOM['cost_variable'] +=df_VOM['cost_fixed']

# c.execute("DELETE FROM CostFixed WHERE tech LIKE 'E_COALSTM_R%' OR tech LIKE 'E_NGACC_R%'")
# c.execute("DELETE FROM CostVariable WHERE tech LIKE 'E_COALSTM_R%' OR tech LIKE 'E_NGACC_R%'")
# df_VOM.drop(columns=['cost_fixed']).to_sql('CostVariable',conn, if_exists='append', index=False, method='multi', chunksize=500)

In [ ]:
if elec_only==1:
    for table in table_list:
        df_cols = c.execute("SELECT * FROM pragma_table_info('" + table[0] + "')").fetchall()
        df_cols = [x[1] for x in df_cols]
        if 'regions' in df_cols:
            query = "DELETE FROM " + table[0] + " WHERE regions NOT IN ('" + "','".join(model_regions) + "' AND regions NOT LIKE '%-%')"
            c.execute(query)

In [ ]:
#add wind integration costs
#https://www.energy.gov/sites/default/files/2021-08/Land-Based%20Wind%20Market%20Report%202021%20Edition_Full%20Report_FINAL.pdf
# wind_intg_costs = np.round(5/3600*1e9/1e6,2) #$5/MWh to $M/PJ

# df_intg_variable = df_costfixed_z[df_costfixed_z['tech'].str.contains('E_WND_N')].copy()
# df_intg_variable.rename(columns={'cost_fixed':'cost_variable', 'cost_fixed_units':'cost_variable_units', \
#                                  'cost_fixed_notes':'cost_variable_notes'}, inplace=True)
# df_intg_variable.loc[:, 'cost_variable_units'] = '$M/PJ'
# df_intg_variable.loc[:, 'cost_variable_notes'] = '$5/MWh integration costs, Based on Land-Based Wind Market Report: 2021 Edition'
# df_intg_variable.loc[:, 'cost_variable'] = wind_intg_costs
# df_intg_variable.to_sql('CostVariable', conn, if_exists='append', index=False)

In [ ]:
#include hydrogen inter-regional transport pipeline data
cwd = os.getcwd()
os.chdir('supply/hydrogen/')
%run integrate_hydrogen_regional_transport.ipynb
os.chdir(cwd)

In [ ]:
#update all sectors regional fuel prices
df_fuelprices = pd.read_csv('supply/AEO_fuel_prices/AEO_2022/reference_case_regional_fuel_prices.csv')
query = 'DELETE FROM CostVariable WHERE tech IN ("' + '", "'.join(df_fuelprices['tech'].unique()) + '")'
c.execute(query)
query = 'DELETE FROM CostVariable WHERE tech = "E_ELCTDLOSS"'
c.execute(query)
df_fuelprices.to_sql('CostVariable',conn, if_exists='append', index=False)

In [ ]:
conn.commit()
conn.close()

In [ ]:
con=sqlite3.connect(outputdB, isolation_level=None)
con.execute("VACUUM")
con.commit()
con.close()

In [ ]:
conn = sqlite3.connect(outputdB)
with open(outputdB.replace('ite',''), 'w') as f:
    for line in conn.iterdump():
        '''
        if 'e+' in line or 'e-0' in line:
            val = [x for x in line.split(',') if 'e+' in x or 'e-0' in x][0]
            ls = line.split(',')
            ind = ls.index(val)
            ls[ind] = [str(round(float(x),2)) for x in line.split(',') if 'e+' in x or 'e-0' in x ][0]
            line = ','.join(ls)
        '''
        if 'CREATE TABLE' in line:
            f.write('\n' )
        f.write('%s\n' % line)
        
conn.close()